
<center><h1>The Annotated Transformer</h1> </center>


<center>
<p><a href="https://arxiv.org/abs/1706.03762">Attention is All You Need
</a></p>
</center>

<img src="images/aiayn.png" width="70%"/>

* *v2022: Austin Huang, Suraj Subramanian, Jonathan Sum, Khalid Almubarak,
   and Stella Biderman.*
* *[Original](https://nlp.seas.harvard.edu/2018/04/03/attention.html):
   [Sasha Rush](http://rush-nlp.com/).*


The Transformer has been on a lot of
people's minds over the last <s>year</s> five years.
This post presents an annotated version of the paper in the
form of a line-by-line implementation. It reorders and deletes
some sections from the original paper and adds comments
throughout. This document itself is a working notebook, and should
be a completely usable implementation.
Code is available
[here](https://github.com/harvardnlp/annotated-transformer/).


<h3> Table of Contents </h3>
<ul>
<li><a href="#prelims">Prelims</a></li>
<li><a href="#background">Background</a></li>
<li><a href="#part-1-model-architecture">Part 1: Model Architecture</a></li>
<li><a href="#model-architecture">Model Architecture</a><ul>
<li><a href="#encoder-and-decoder-stacks">Encoder and Decoder Stacks</a></li>
<li><a href="#position-wise-feed-forward-networks">Position-wise Feed-Forward
Networks</a></li>
<li><a href="#embeddings-and-softmax">Embeddings and Softmax</a></li>
<li><a href="#positional-encoding">Positional Encoding</a></li>
<li><a href="#full-model">Full Model</a></li>
<li><a href="#inference">Inference:</a></li>
</ul></li>
<li><a href="#part-2-model-training">Part 2: Model Training</a></li>
<li><a href="#training">Training</a><ul>
<li><a href="#batches-and-masking">Batches and Masking</a></li>
<li><a href="#training-loop">Training Loop</a></li>
<li><a href="#training-data-and-batching">Training Data and Batching</a></li>
<li><a href="#hardware-and-schedule">Hardware and Schedule</a></li>
<li><a href="#optimizer">Optimizer</a></li>
<li><a href="#regularization">Regularization</a></li>
</ul></li>
<li><a href="#a-first-example">A First Example</a><ul>
<li><a href="#synthetic-data">Synthetic Data</a></li>
<li><a href="#loss-computation">Loss Computation</a></li>
<li><a href="#greedy-decoding">Greedy Decoding</a></li>
</ul></li>
<li><a href="#part-3-a-real-world-example">Part 3: A Real World Example</a>
<ul>
<li><a href="#data-loading">Data Loading</a></li>
<li><a href="#iterators">Iterators</a></li>
<li><a href="#training-the-system">Training the System</a></li>
</ul></li>
<li><a href="#additional-components-bpe-search-averaging">Additional
Components: BPE, Search, Averaging</a></li>
<li><a href="#results">Results</a><ul>
<li><a href="#attention-visualization">Attention Visualization</a></li>
<li><a href="#encoder-self-attention">Encoder Self Attention</a></li>
<li><a href="#decoder-self-attention">Decoder Self Attention</a></li>
<li><a href="#decoder-src-attention">Decoder Src Attention</a></li>
</ul></li>
<li><a href="#conclusion">Conclusion</a></li>
</ul>

# Prelims

<a href="#background">Skip</a>

In [ ]:
# !pip install -r requirements.txt

In [ ]:
# # Uncomment for colab
# #
# !pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil
# !python -m spacy download de_core_news_sm
# !python -m spacy download en_core_web_sm

In [ ]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [ ]:
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

> My comments are blockquoted. The main text is all from the paper itself.

# Background


The goal of reducing sequential computation also forms the
foundation of the Extended Neural GPU, ByteNet and ConvS2S, all of
which use convolutional neural networks as basic building block,
computing hidden representations in parallel for all input and
output positions. In these models, the number of operations required
to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and
logarithmically for ByteNet. This makes it more difficult to learn
dependencies between distant positions. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of
reduced effective resolution due to averaging attention-weighted
positions, an effect we counteract with Multi-Head Attention.

Self-attention, sometimes called intra-attention is an attention
mechanism relating different positions of a single sequence in order
to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading
comprehension, abstractive summarization, textual entailment and
learning task-independent sentence representations. End-to-end
memory networks are based on a recurrent attention mechanism instead
of sequencealigned recurrence and have been shown to perform well on
simple-language question answering and language modeling tasks.

To the best of our knowledge, however, the Transformer is the first
transduction model relying entirely on self-attention to compute
representations of its input and output without using sequence
aligned RNNs or convolution.

# Part 1: Model Architecture

# Model Architecture


Most competitive neural sequence transduction models have an
encoder-decoder structure
[(cite)](https://arxiv.org/abs/1409.0473). Here, the encoder maps an
input sequence of symbol representations $(x_1, ..., x_n)$ to a
sequence of continuous representations $\mathbf{z} = (z_1, ...,
z_n)$. Given $\mathbf{z}$, the decoder then generates an output
sequence $(y_1,...,y_m)$ of symbols one element at a time. At each
step the model is auto-regressive
[(cite)](https://arxiv.org/abs/1308.0850), consuming the previously
generated symbols as additional input when generating the next.

In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

这段代码定义了一个标准的编码器-解码器（Encoder-Decoder）架构，作为其他模型的基础。

在初始化函数中，`EncoderDecoder` 类接受五个参数：
- `encoder`：编码器模型。
- `decoder`：解码器模型。
- `src_embed`：源语言嵌入层。
- `tgt_embed`：目标语言嵌入层。
- `generator`：生成器（输出层）。

在初始化过程中，通过调用 `super(EncoderDecoder, self).__init__()` 构造父类的初始化函数。

在前向传播函数中，输入包括源语言输入序列 `src`、目标语言输入序列 `tgt`，以及对应的掩码 `src_mask` 和 `tgt_mask`。

- `encode` 方法：通过调用源语言嵌入层 `self.src_embed(src)` 将源语言输入序列 `src` 进行嵌入操作，然后通过编码器 `self.encoder` 对嵌入结果进行编码，得到编码器的输出 `memory`。
- `decode` 方法：通过调用目标语言嵌入层 `self.tgt_embed(tgt)` 将目标语言输入序列 `tgt` 进行嵌入操作，然后通过解码器 `self.decoder` 对嵌入结果进行解码，得到解码器的输出。
- `forward` 方法：调用 `encode` 方法将源语言输入序列编码为 `memory`，然后调用 `decode` 方法将 `memory`、源语言掩码 `src_mask`、目标语言输入序列 `tgt` 和目标语言掩码 `tgt_mask` 传递给解码器，得到最终的解码结果。

整体而言，这段代码定义了一个标准的编码器-解码器架构。它将编码器、解码器、嵌入层和生成器组合在一起，提供了对源语言和目标语言序列进行编码和解码的接口。编码器将源语言输入序列进行编码，生成编码器的输出 `memory`，解码器使用 `memory`、源语言掩码、目标语言输入序列和目标语言掩码进行解码，得到最终的解码结果。这段代码定义了一个标准的编码器-解码器（Encoder-Decoder）架构，作为其他模型的基础。

在初始化函数中，`EncoderDecoder` 类接受五个参数：
- `encoder`：编码器模型。
- `decoder`：解码器模型。
- `src_embed`：源语言嵌入层。
- `tgt_embed`：目标语言嵌入层。
- `generator`：生成器（输出层）。

在初始化过程中，通过调用 `super(EncoderDecoder, self).__init__()` 构造父类的初始化函数。

在前向传播函数中，输入包括源语言输入序列 `src`、目标语言输入序列 `tgt`，以及对应的掩码 `src_mask` 和 `tgt_mask`。

- `encode` 方法：通过调用源语言嵌入层 `self.src_embed(src)` 将源语言输入序列 `src` 进行嵌入操作，然后通过编码器 `self.encoder` 对嵌入结果进行编码，得到编码器的输出 `memory`。
- `decode` 方法：通过调用目标语言嵌入层 `self.tgt_embed(tgt)` 将目标语言输入序列 `tgt` 进行嵌入操作，然后通过解码器 `self.decoder` 对嵌入结果进行解码，得到解码器的输出。
- `forward` 方法：调用 `encode` 方法将源语言输入序列编码为 `memory`，然后调用 `decode` 方法将 `memory`、源语言掩码 `src_mask`、目标语言输入序列 `tgt` 和目标语言掩码 `tgt_mask` 传递给解码器，得到最终的解码结果。

整体而言，这段代码定义了一个标准的编码器-解码器架构。它将编码器、解码器、嵌入层和生成器组合在一起，提供了对源语言和目标语言序列进行编码和解码的接口。编码器将源语言输入序列进行编码，生成编码器的输出 `memory`，解码器使用 `memory`、源语言掩码、目标语言输入序列和目标语言掩码进行解码，得到最终的解码结果。

In [ ]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

这段代码定义了一个生成器（Generator）类，用于在Transformer模型中进行线性变换和softmax操作以生成输出。

在初始化函数中，生成器接受两个参数：d_model 表示输入向量的维度，vocab 表示输出的词汇表大小。

在前向传播函数中，输入 x 是一个表示模型的隐藏状态或特征的张量。通过对输入进行线性变换，通过 self.proj 层，将输入从隐藏维度（d_model）映射到输出词汇表大小的维度。

然后，通过对线性变换后的结果应用 log_softmax 函数，并指定 dim=-1 来计算输出的对数softmax值。dim=-1 表示对最后一个维度进行softmax计算，即在这里是对输出词汇表的维度进行softmax。

最后，将计算得到的对数softmax值作为函数的返回结果。

综上所述，这段代码定义了一个简单的生成器类，它通过线性变换将输入映射到输出词汇表的维度，并使用softmax函数计算输出的对数概率值。这是Transformer模型中生成输出的一部分。


The Transformer follows this overall architecture using stacked
self-attention and point-wise, fully connected layers for both the
encoder and decoder, shown in the left and right halves of Figure 1,
respectively.

![](images/ModalNet-21.png)

## Encoder and Decoder Stacks

### Encoder

The encoder is composed of a stack of $N=6$ identical layers.

In [ ]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

这段代码定义了一个编码器（Encoder）模块，它由多个相同类型的层堆叠而成。

在初始化函数中，`Encoder` 类接受两个参数：`layer` 表示编码器中的单个层（Layer），`N` 表示层的数量。它通过调用父类 `nn.Module` 的 `__init__` 方法进行初始化，并使用 `clones` 函数复制 `layer` N 次，得到一个层的列表 `self.layers`。同时，创建了一个层归一化（LayerNorm）模块 `self.norm`，用于对编码器输出进行归一化。

在前向传播函数中，输入 `x` 是编码器的输入张量，`mask` 是一个用于掩码的张量（可选）。它通过将输入张量 `x` 逐层传递给编码器中的每个层进行处理。

具体来说，通过迭代遍历 `self.layers` 列表中的每个层，并将输入张量 `x` 传递给每个层进行计算。在每个层的计算过程中，可能还会使用 `mask` 对输入进行掩码处理。

最后，通过 `self.norm(x)` 对编码器输出进行层归一化，将每个维度的特征进行归一化操作，以提高模型的训练稳定性和表示能力。

综上所述，这段代码定义了一个编码器模块，它由多个层堆叠而成。在前向传播过程中，输入通过每个层进行逐层处理，并最终通过层归一化获得编码器的输出。编码器常用于Transformer模型等中，用于对输入序列进行特征提取和编码。


We employ a residual connection
[(cite)](https://arxiv.org/abs/1512.03385) around each of the two
sub-layers, followed by layer normalization
[(cite)](https://arxiv.org/abs/1607.06450).

In [ ]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

这段代码定义了一个 Layer Normalization（层归一化）的模块。

在初始化函数中，`LayerNorm` 类接受两个参数：`features` 表示输入的特征维度，`eps` 是一个小的常数，用于防止除零错误。它通过调用父类 `nn.Module` 的 `__init__` 方法进行初始化，并定义了两个可学习的参数 `self.a_2` 和 `self.b_2`，它们分别对应于缩放（scale）和偏移（shift）操作。

在前向传播函数中，输入 `x` 是一个表示模型的隐藏状态或特征的张量。通过对输入进行计算，实现了层归一化的操作。

具体来说，首先计算输入 `x` 沿着最后一个维度的均值 `mean` 和标准差 `std`。这里使用了 `x.mean(-1, keepdim=True)` 和 `x.std(-1, keepdim=True)` 来计算均值和标准差，并保持维度一致。

然后，通过 `(x - mean) / (std + self.eps)` 对输入进行归一化操作，将输入减去均值，除以标准差。这里使用了广播机制，确保计算在每个元素上都可以正确进行。

最后，通过 `self.a_2 * (x - mean) / (std + self.eps) + self.b_2` 将归一化后的结果进行缩放和偏移，得到最终的层归一化结果。

综上所述，这段代码定义了一个层归一化的模块，它对输入进行归一化操作，以及可学习的缩放和偏移操作。层归一化在神经网络中用于提高模型的训练稳定性和泛化性能，常用于Transformer等模型中的每个子层的输入。


That is, the output of each sub-layer is $\mathrm{LayerNorm}(x +
\mathrm{Sublayer}(x))$, where $\mathrm{Sublayer}(x)$ is the function
implemented by the sub-layer itself.  We apply dropout
[(cite)](http://jmlr.org/papers/v15/srivastava14a.html) to the
output of each sub-layer, before it is added to the sub-layer input
and normalized.

To facilitate these residual connections, all sub-layers in the
model, as well as the embedding layers, produce outputs of dimension
$d_{\text{model}}=512$.

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

这段代码定义了一个子层连接（SublayerConnection）模块，它包含一个残差连接（residual connection）和一个层归一化（LayerNorm）操作。

在初始化函数中，`SublayerConnection` 类接受两个参数：`size` 表示输入的特征维度大小，`dropout` 是一个用于随机失活的概率。它通过调用父类 `nn.Module` 的 `__init__` 方法进行初始化，并创建了一个层归一化模块 `self.norm` 和一个 dropout 模块 `self.dropout`。

在前向传播函数中，输入 `x` 是一个张量，表示来自上一层的输入。`sublayer` 是一个函数或模块，代表子层的计算过程。该函数或模块的输入和输出维度与 `x` 相同。

具体来说，`forward` 函数实现了将残差连接应用于任意子层的操作。它首先对输入 `x` 进行层归一化操作，通过 `self.norm(x)` 得到归一化的结果。

然后，通过将归一化后的结果传递给 `sublayer` 进行计算，并应用 dropout 操作，即 `self.dropout(sublayer(self.norm(x)))`。

最后，通过将计算结果与输入 `x` 相加，实现残差连接，即 `x + self.dropout(sublayer(self.norm(x)))`。这里的加法操作将输入 `x` 与经过子层计算和残差连接后的结果相加，从而将子层的计算结果与原始输入进行融合。

综上所述，这段代码定义了一个子层连接模块，它实现了残差连接和层归一化操作，用于将子层的计算结果与输入进行融合。这种设计常用于Transformer模型中的编码器和解码器中，以促进信息流动和提高模型性能。

具体来说，`sublayer` 是一个表示子层的函数或模块，它将输入 `x` 经过一系列的计算得到子层的输出。而 `self.dropout` 是一个 dropout 模块，用于在训练过程中随机地将一部分元素置为零，以防止模型过拟合。

在这里，通过应用 `self.dropout` 到 `sublayer(self.norm(x))`，实际上是对子层的计算结果进行了随机失活操作。dropout 操作以一定的概率将子层计算结果的某些元素置为零，从而减少模型对于特定元素的依赖，防止过拟合，并提高模型的泛化能力。

需要注意的是，这段代码中的 `self.dropout` 是在 `SublayerConnection` 模块的初始化函数中定义的，它会在每次前向传播过程中被调用。因此，在每个子层的计算结果上应用 dropout 操作，可以在每个前向传播步骤中以一定的概率随机丢弃一些计算结果的元素。

综上所述，通过在子层的计算结果上应用 `self.dropout`，可以对子层的输出进行随机失活操作，以增强模型的鲁棒性和泛化能力。这对于防止过拟合并提高模型性能非常有用。


Each layer has two sub-layers. The first is a multi-head
self-attention mechanism, and the second is a simple, position-wise
fully connected feed-forward network.

In [ ]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"

    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

这段代码定义了编码器层（EncoderLayer）模块，它由自注意力机制和前馈神经网络组成。

在初始化函数中，`EncoderLayer` 类接受四个参数：`size` 表示编码器层的输入和输出的特征维度大小，`self_attn` 表示自注意力机制的模块，`feed_forward` 表示前馈神经网络的模块，`dropout` 是一个用于随机失活的概率。它通过调用父类 `nn.Module` 的 `__init__` 方法进行初始化，并创建了两个 `SublayerConnection` 的实例 `self.sublayer`。

在前向传播函数中，输入 `x` 是编码器层的输入张量，`mask` 是一个用于掩码的张量。按照图示中的连接方式，首先将输入 `x` 传递给第一个子层连接 `self.sublayer[0]`，其中包含了自注意力机制。在这个子层连接中，通过调用 `self.self_attn(x, x, x, mask)` 来对输入进行自注意力计算，获取编码器层的中间输出。

然后，将中间输出传递给第二个子层连接 `self.sublayer[1]`，其中包含了前馈神经网络。在这个子层连接中，通过调用 `self.feed_forward` 对中间输出进行前馈神经网络的计算，得到编码器层的最终输出。

最后，将最终输出作为函数的返回结果。

综上所述，这段代码定义了一个编码器层模块，它由自注意力机制和前馈神经网络组成。在前向传播过程中，输入经过自注意力计算和前馈神经网络计算，以获取编码器层的输出。这种设计常用于Transformer模型的编码器中，用于对输入序列进行特征提取和编码。

### Decoder

The decoder is also composed of a stack of $N=6$ identical layers.


In [ ]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."

    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

这段代码定义了一个解码器（Decoder）模块，它由多个相同类型的层堆叠而成，并带有掩码（masking）操作。

在初始化函数中，`Decoder` 类接受两个参数：`layer` 表示解码器中的单个层（Layer），`N` 表示层的数量。它通过调用父类 `nn.Module` 的 `__init__` 方法进行初始化，并使用 `clones` 函数复制 `layer` N 次，得到一个层的列表 `self.layers`。同时，创建了一个层归一化（LayerNorm）模块 `self.norm`，用于对解码器输出进行归一化。

在前向传播函数中，输入 `x` 是解码器的输入张量，`memory` 是编码器的输出张量（记忆张量），`src_mask` 是一个用于对编码器输出进行掩码的张量，`tgt_mask` 是一个用于对解码器输入进行掩码的张量。

具体来说，通过迭代遍历 `self.layers` 列表中的每个层，并将输入张量 `x`、记忆张量 `memory`、编码器输出的掩码 `src_mask`、解码器输入的掩码 `tgt_mask` 传递给每个层进行处理。

在每个层的计算过程中，会利用 `memory` 中的编码器输出作为参考进行注意力计算，使用 `src_mask` 控制对编码器输出的注意力分布，并使用 `tgt_mask` 控制对解码器输入的注意力分布。

最后，通过 `self.norm(x)` 对解码器的输出进行层归一化，将每个维度的特征进行归一化操作，以提高模型的训练稳定性和表示能力。

综上所述，这段代码定义了一个解码器模块，它由多个层堆叠而成，并带有掩码操作。在前向传播过程中，输入通过每个层进行逐层处理，并最终通过层归一化获得解码器的输出。解码器常用于Transformer模型中，用于将编码器的输出解码为目标序列。


In addition to the two sub-layers in each encoder layer, the decoder
inserts a third sub-layer, which performs multi-head attention over
the output of the encoder stack.  Similar to the encoder, we employ
residual connections around each of the sub-layers, followed by
layer normalization.

In [ ]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

这段代码定义了一个解码器层（DecoderLayer）模块，它由自注意力机制、源注意力机制和前馈神经网络组成。

在初始化函数中，`DecoderLayer` 类接受五个参数：`size` 表示解码器层的输入和输出的特征维度大小，`self_attn` 表示自注意力机制的模块，`src_attn` 表示源注意力机制的模块，`feed_forward` 表示前馈神经网络的模块，`dropout` 是一个用于随机失活的概率。它通过调用父类 `nn.Module` 的 `__init__` 方法进行初始化，并创建了三个 `SublayerConnection` 的实例 `self.sublayer`。

在前向传播函数中，输入 `x` 是解码器层的输入张量，`memory` 是编码器的输出张量（记忆张量），`src_mask` 是一个用于对编码器输出进行掩码的张量，`tgt_mask` 是一个用于对解码器输入进行掩码的张量。

具体来说，根据图示中的连接方式，首先将输入 `x` 传递给第一个子层连接 `self.sublayer[0]`，其中包含了自注意力机制。在这个子层连接中，通过调用 `self.self_attn(x, x, x, tgt_mask)` 来对输入进行自注意力计算，获取解码器层的中间输出。

然后，将中间输出传递给第二个子层连接 `self.sublayer[1]`，其中包含了源注意力机制。在这个子层连接中，通过调用 `self.src_attn(x, m, m, src_mask)` 来对中间输出和记忆张量进行注意力计算，得到解码器层的中间输出。

最后，将中间输出传递给第三个子层连接 `self.sublayer[2]`，其中包含了前馈神经网络。在这个子层连接中，通过调用 `self.feed_forward` 对中间输出进行前馈神经网络的计算，得到解码器层的最终输出。

综上所述，这段代码定义了一个解码器层模块，它由自注意力机制、源注意力机制和前馈神经网络组成。在前向传播过程中，输入经过自注意力计算、源注意力计算和前馈神经网络计算，以获取解码器层的输出。这种设计常用于Transformer模型的解码器中，用于将编码器的输出解码为目标序列。


We also modify the self-attention sub-layer in the decoder stack to
prevent positions from attending to subsequent positions.  This
masking, combined with fact that the output embeddings are offset by
one position, ensures that the predictions for position $i$ can
depend only on the known outputs at positions less than $i$.

In [ ]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

这段代码定义了一个函数 `subsequent_mask(size)`，用于生成一个遮蔽矩阵，用于在自注意力机制中屏蔽后续位置的信息。

函数接受一个参数 `size`，表示生成遮蔽矩阵的大小。

具体来说，首先通过 `(1, size, size)` 定义了遮蔽矩阵的形状 `attn_shape`，其中 `1` 表示批次大小为1（因为遮蔽矩阵在每个样本上都是相同的），`size` 表示遮蔽矩阵的高度和宽度。

然后，通过 `torch.triu(torch.ones(attn_shape), diagonal=1)` 生成了一个上三角矩阵，该矩阵的主对角线及其以下的元素为1，其余元素为0。这样的矩阵正好可以用于屏蔽后续位置的信息。

接下来，通过 `.type(torch.uint8)` 将矩阵的数据类型转换为无符号整型，以便在后续的逻辑中进行逻辑比较。

最后，通过 `return subsequent_mask == 0` 将遮蔽矩阵与0进行逻辑比较，得到一个由True和False组成的布尔型遮蔽矩阵，其中True表示需要被遮蔽的位置，False表示不需要被遮蔽的位置。

综上所述，这段代码定义了一个函数，用于生成一个遮蔽矩阵，该遮蔽矩阵可以在自注意力机制中屏蔽后续位置的信息，以防止当前位置获取未来位置的信息。这在Transformer模型等序列生成任务中非常有用。


> Below the attention mask shows the position each tgt word (row) is
> allowed to look at (column). Words are blocked for attending to
> future words during training.

In [ ]:
def example_mask():
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Subsequent Mask": subsequent_mask(20)[0][x, y].flatten(),
                    "Window": y,
                    "Masking": x,
                }
            )
            for y in range(20)
            for x in range(20)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect()
        .properties(height=250, width=250)
        .encode(
            alt.X("Window:O"),
            alt.Y("Masking:O"),
            alt.Color("Subsequent Mask:Q", scale=alt.Scale(scheme="viridis")),
        )
        .interactive()
    )


show_example(example_mask)

这段代码展示了如何使用 `subsequent_mask` 函数生成遮蔽矩阵，并将其可视化。

首先，通过使用列表推导式生成了一个 Pandas DataFrame `LS_data`，其中包含了遮蔽矩阵的各个元素以及相应的行列索引信息。具体地，`y` 和 `x` 都遍历了 0 到 19 的范围，用于生成 20x20 的遮蔽矩阵。对于每个位置，将 `subsequent_mask(20)[0][x, y].flatten()` 的值作为 "Subsequent Mask" 列的元素，将 `y` 的值作为 "Window" 列的元素，将 `x` 的值作为 "Masking" 列的元素。最终，通过 `pd.concat` 将生成的 DataFrame 连接起来。

`subsequent_mask(20)[0][x, y].flatten()`  这一句代码的含义是获取生成的遮蔽矩阵 `subsequent_mask(20)` 的某个位置上的元素值。首先，`subsequent_mask(20)` 调用了之前定义的 `subsequent_mask` 函数，并传入参数 `20`，生成一个遮蔽矩阵。这个遮蔽矩阵具有 `20x20` 的形状。然后，通过 `[0][x, y]` 的索引操作，从遮蔽矩阵的第一个元素（第一维）中，获取指定位置 `(x, y)` 处的元素。这里的 `x` 和 `y` 是之前在列表推导式中循环遍历的变量，代表遮蔽矩阵中的行和列索引。最后，通过 `.flatten()` 方法将获取的元素值展平为一维数组。这样做是为了方便后续使用，将遮蔽矩阵的元素值作为 "Subsequent Mask" 列的元素之一。这一句代码的含义是从生成的遮蔽矩阵中获取指定位置 `(x, y)` 处的元素值，并将其展平为一维数组。

接下来，使用 Altair 库创建了一个图表对象。使用 `.mark_rect()` 指定图表中的图形类型为矩形，`.properties(height=250, width=250)` 设置图表的高度和宽度为 250 像素。然后，使用 `.encode()` 方法指定了数据在图表中的编码方式。`alt.X("Window:O")` 将 "Window" 列作为 x 轴，`alt.Y("Masking:O")` 将 "Masking" 列作为 y 轴，`alt.Color("Subsequent Mask:Q", scale=alt.Scale(scheme="viridis"))` 将 "Subsequent Mask" 列作为颜色编码，并使用 "viridis" 调色板进行着色。

最后，通过 `.interactive()` 使图表具有交互性。

函数 `show_example()` 被调用，用于显示生成的图表对象 `example_mask()`。

综上所述，这段代码使用 `subsequent_mask` 函数生成遮蔽矩阵，并使用 Altair 库将其可视化为一个交互式的矩形图表，用于展示遮蔽矩阵中的不同位置和遮蔽情况。

### Attention

An attention function can be described as mapping a query and a set
of key-value pairs to an output, where the query, keys, values, and
output are all vectors.  The output is computed as a weighted sum of
the values, where the weight assigned to each value is computed by a
compatibility function of the query with the corresponding key.

We call our particular attention "Scaled Dot-Product Attention".
The input consists of queries and keys of dimension $d_k$, and
values of dimension $d_v$.  We compute the dot products of the query
with all keys, divide each by $\sqrt{d_k}$, and apply a softmax
function to obtain the weights on the values.



![](images/ModalNet-19.png)


In practice, we compute the attention function on a set of queries
simultaneously, packed together into a matrix $Q$.  The keys and
values are also packed together into matrices $K$ and $V$.  We
compute the matrix of outputs as:

$$
   \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

这段代码定义了一个函数 `attention`，用于计算缩放点积注意力（Scaled Dot Product Attention）。

函数接受五个参数：
- `query`: 查询张量，形状为 [batch_size, query_len, d_k]。
- `key`: 键张量，形状为 [batch_size, key_len, d_k]。
- `value`: 值张量，形状为 [batch_size, value_len, d_v]。
- `mask`: 用于掩码的张量，形状为 [batch_size, query_len, key_len]，可以将特定位置的注意力分数屏蔽为负无穷大。
- `dropout`: 可选的 dropout 模块，用于对注意力分布进行随机失活操作。

具体来说，首先通过 `query.size(-1)` 获取查询张量的最后一个维度 `d_k`，即特征维度。

然后，通过 `torch.matmul(query, key.transpose(-2, -1))` 进行点积操作，并除以 `math.sqrt(d_k)` 进行缩放，得到注意力分数张量 `scores`。此处使用了张量的转置操作 `key.transpose(-2, -1)`，将键张量的倒数第二个和倒数第一个维度进行转置，以适配点积运算。一般情况下， key的倒数第一维是向量的d_k, 倒数第二维是输入key的sequence的长度。

接下来，如果存在掩码 `mask`，则使用 `scores.masked_fill(mask == 0, -1e9)` 将掩码中为0的位置对应的注意力分数置为负无穷大，以实现屏蔽效果。

然后，通过 `scores.softmax(dim=-1)` 对注意力分数进行 softmax 操作，得到注意力权重张量 `p_attn`。注意力权重表示了每个位置的重要性权重。

最后，如果提供了 `dropout`，则对注意力权重进行随机失活操作，即 `dropout(p_attn)`。

函数返回通过注意力权重加权后的值张量 `torch.matmul(p_attn, value)`，以及注意力权重张量 `p_attn`。

其中 `p_attn` 是注意力权重张量，其形状取决于输入的查询张量和键张量的形状。通常情况下，如果查询张量的形状是 [batch_size, query_len, d_k]，键张量的形状是 [batch_size, key_len, d_k]，那么注意力权重张量 `p_attn` 的形状为 [batch_size, query_len, key_len]。
每个维度的含义如下：
- 第一个维度 `batch_size`：表示批次的大小，即处理的样本数量。
- 第二个维度 `query_len`：表示查询张量的长度，即查询序列的长度。
- 第三个维度 `key_len`：表示键张量的长度，即键序列的长度。
注意力权重张量 `p_attn` 中的每个元素表示了查询序列中的每个位置与键序列中的每个位置之间的注意力权重。它描述了查询序列中每个位置对键序列中的位置的重要性权重。可以通过索引 `p_attn[i, j, k]` 获取批次中第 `i` 个样本的查询位置 `j` 对键位置 `k` 的注意力权重值。`p_attn` 的形状为 [batch_size, query_len, key_len]，其中每个维度表示了对应的含义。

综上所述，这段代码定义了一个用于计算缩放点积注意力的函数。它通过将查询、键和值张量进行点积运算，并使用缩放因子进行缩放，得到注意力分数。然后，根据掩码进行屏蔽操作，并通过 softmax 函数得到注意力权重。最后，对注意力权重进行可选的随机失活操作，并返回通过注意力权重加权后的值张量和注意力权重张量。


The two most commonly used attention functions are additive
attention [(cite)](https://arxiv.org/abs/1409.0473), and dot-product
(multiplicative) attention.  Dot-product attention is identical to
our algorithm, except for the scaling factor of
$\frac{1}{\sqrt{d_k}}$. Additive attention computes the
compatibility function using a feed-forward network with a single
hidden layer.  While the two are similar in theoretical complexity,
dot-product attention is much faster and more space-efficient in
practice, since it can be implemented using highly optimized matrix
multiplication code.


While for small values of $d_k$ the two mechanisms perform
similarly, additive attention outperforms dot product attention
without scaling for larger values of $d_k$
[(cite)](https://arxiv.org/abs/1703.03906). We suspect that for
large values of $d_k$, the dot products grow large in magnitude,
pushing the softmax function into regions where it has extremely
small gradients (To illustrate why the dot products get large,
assume that the components of $q$ and $k$ are independent random
variables with mean $0$ and variance $1$.  Then their dot product,
$q \cdot k = \sum_{i=1}^{d_k} q_ik_i$, has mean $0$ and variance
$d_k$.). To counteract this effect, we scale the dot products by
$\frac{1}{\sqrt{d_k}}$.



![](images/ModalNet-20.png)


Multi-head attention allows the model to jointly attend to
information from different representation subspaces at different
positions. With a single attention head, averaging inhibits this.

$$
\mathrm{MultiHead}(Q, K, V) =
    \mathrm{Concat}(\mathrm{head_1}, ..., \mathrm{head_h})W^O \\
    \text{where}~\mathrm{head_i} = \mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i)
$$

Where the projections are parameter matrices $W^Q_i \in
\mathbb{R}^{d_{\text{model}} \times d_k}$, $W^K_i \in
\mathbb{R}^{d_{\text{model}} \times d_k}$, $W^V_i \in
\mathbb{R}^{d_{\text{model}} \times d_v}$ and $W^O \in
\mathbb{R}^{hd_v \times d_{\text{model}}}$.

In this work we employ $h=8$ parallel attention layers, or
heads. For each of these we use $d_k=d_v=d_{\text{model}}/h=64$. Due
to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full
dimensionality.

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        # 3) "Concat" using a view and apply a final linear.
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x)

这段代码定义了一个多头注意力（MultiHeadedAttention）模块，用于实现 Transformer 模型中的多头自注意力机制。

在初始化函数中，`MultiHeadedAttention` 类接受三个参数：
- `h`：表示头的数量，即多头注意力的个数。
- `d_model`：表示输入和输出的特征维度。
- `dropout`：可选参数，表示用于随机失活的概率。

在初始化过程中，通过断言 `assert d_model % h == 0` 来确保特征维度 `d_model` 能够被头的数量 `h` 整除。接着，定义了每个头的特征维度 `self.d_k`，即 `d_model` 除以 `h` 的结果。然后，使用 `clones` 函数复制了四个线性层 `nn.Linear(d_model, d_model)`，分别用于处理查询、键、值和最终的线性变换。另外，还定义了一个用于进行随机失活的 `dropout` 模块。

在前向传播函数中，输入包括 `query`、`key`、`value` 和可选的掩码 `mask`。首先，如果存在掩码 `mask`，则通过 `mask.unsqueeze(1)` 将其在第二维上进行扩展，以适应多头注意力的操作。

接下来，根据论文中的步骤，执行以下操作：

1) 执行线性变换，将 `query`、`key` 和 `value` 分别传入对应的线性层，并对每个线性层的输出进行维度变换，使其变为形状为 `[nbatches, seq_len, h, d_k]` 的张量。其中，`nbatches` 表示批次大小，`seq_len` 表示序列长度。

2) 对变换后的张量应用注意力操作，调用 `attention` 函数。这里的 `attention` 函数就是之前解析的注意力计算函数，用于计算多头注意力的输出张量 `x` 和注意力权重张量 `self.attn`。

3) 将输出张量 `x` 进行维度变换，将其转置为形状为 `[nbatches, seq_len, h * d_k]` 的连续张量。

最后，通过调用 `self.linears[-1]` 对转置后的张量进行线性变换，得到最终的输出。

整体而言，这段代码实现了多头注意力机制。它首先对查询、键和值进行线性变换和维度变换，然后应用注意力操作，最后通过线性变换得到输出。多头注意力可以提升模型的表示能力，允许模型在不同的表示子空间上进行并行计算，并且每个头可以关注序列中不同位置的相关信息。

**注意， 下面这段代码具体是这样执行的**

```python
    query, key, value = [
        lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
        for lin, x in zip(self.linears, (query, key, value))
    ]
```
这段代码用于将批量的查询（query）、键（key）和值（value）矩阵从特征维度 `d_model` 投影为多头注意力中的形状 `h * d_k`。

具体而言，代码通过遍历三个线性层 `self.linears` 和三个输入张量 `(query, key, value)`，分别进行以下操作：

1) 对每个输入张量 `x`，通过线性变换 `lin(x)` 得到投影后的结果。

2) 使用 `.view(nbatches, -1, self.h, self.d_k)` 将投影结果进行形状重塑，将特征维度 `d_model` 分成 `h` 个头 `self.h`，每个头的特征维度为 `d_k`。

3) 最后，通过 `.transpose(1, 2)` 进行维度转置，将头维度 `self.h` 和序列维度 `seq_len` 进行交换，以满足多头注意力的计算需求。

经过投影之后，`query`、`key` 和 `value` 的形状会发生变化。假设原始输入的形状为：
- `query`: [batch_size, query_len, d_model]
- `key`: [batch_size, key_len, d_model]
- `value`: [batch_size, value_len, d_model]

经过投影后，它们的形状变为：
- `query`: [batch_size,h, query_len, d_k]
- `key`: [batch_size, h, key_len, d_k]
- `value`: [batch_size, h, value_len, d_k]

其中，
- `batch_size` 表示批次大小，即处理的样本数量。
- `query_len` 表示查询序列的长度。
- `key_len` 表示键序列的长度。
- `value_len` 表示值序列的长度。
- `h` 表示头的数量，即多头注意力的个数。
- `d_k` 表示每个头的特征维度。

投影之后，每个输入矩阵在特征维度 `d_model` 上被分为 `h` 个头，每个头的特征维度为 `d_k`。维度变换后，头维度 `h` 位于第三个维度上，即在形状中的索引位置为 2。这样的形状安排使得后续的多头注意力计算可以并行处理不同的头。

综上所述，经过投影后的 `query`、`key` 和 `value` 是具有多头注意力形状的张量。每个矩阵都包含了多个头，每个头具有特定的特征维度。

总结起来，这段代码通过线性变换和形状重塑的操作，将输入的查询、键和值矩阵从特征维度 `d_model` 投影为多头注意力中的形状 `h * d_k`，为接下来的多头注意力计算做准备。每个头的特征维度为 `d_k`，总共有 `h` 个头。通过维度转置，确保输入张量在头和序列维度上的顺序满足多头注意力计算的要求。

### Applications of Attention in our Model

The Transformer uses multi-head attention in three different ways:
1) In "encoder-decoder attention" layers, the queries come from the
previous decoder layer, and the memory keys and values come from the
output of the encoder.  This allows every position in the decoder to
attend over all positions in the input sequence.  This mimics the
typical encoder-decoder attention mechanisms in sequence-to-sequence
models such as [(cite)](https://arxiv.org/abs/1609.08144).


2) The encoder contains self-attention layers.  In a self-attention
layer all of the keys, values and queries come from the same place,
in this case, the output of the previous layer in the encoder.  Each
position in the encoder can attend to all positions in the previous
layer of the encoder.


3) Similarly, self-attention layers in the decoder allow each
position in the decoder to attend to all positions in the decoder up
to and including that position.  We need to prevent leftward
information flow in the decoder to preserve the auto-regressive
property.  We implement this inside of scaled dot-product attention
by masking out (setting to $-\infty$) all values in the input of the
softmax which correspond to illegal connections.

## Position-wise Feed-Forward Networks

In addition to attention sub-layers, each of the layers in our
encoder and decoder contains a fully connected feed-forward network,
which is applied to each position separately and identically.  This
consists of two linear transformations with a ReLU activation in
between.

$$\mathrm{FFN}(x)=\max(0, xW_1 + b_1) W_2 + b_2$$

While the linear transformations are the same across different
positions, they use different parameters from layer to
layer. Another way of describing this is as two convolutions with
kernel size 1.  The dimensionality of input and output is
$d_{\text{model}}=512$, and the inner-layer has dimensionality
$d_{ff}=2048$.

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

这段代码定义了一个位置前馈神经网络（Positionwise Feed-Forward）模块，用于实现 Transformer 模型中的前馈神经网络层。

在初始化函数中，`PositionwiseFeedForward` 类接受三个参数：
- `d_model`：表示输入和输出的特征维度。
- `d_ff`：表示前馈神经网络隐藏层的特征维度。
- `dropout`：可选参数，表示用于随机失活的概率。

在初始化过程中，通过两个线性层 `nn.Linear(d_model, d_ff)` 和 `nn.Linear(d_ff, d_model)` 来构建前馈神经网络的两个层。其中，第一个线性层将输入特征维度 `d_model` 转换为隐藏层特征维度 `d_ff`，第二个线性层将隐藏层特征维度 `d_ff` 转换回输入特征维度 `d_model`。另外，还定义了一个用于随机失活的 `dropout` 模块。

在前向传播函数中，输入为 `x`，表示经过自注意力层处理后的张量。首先，通过 `self.w_1(x)` 将输入张量 `x` 输入到第一个线性层中，然后应用 ReLU 激活函数。接着，通过 `self.dropout` 对激活后的张量进行随机失活操作。最后，将随机失活后的张量输入到第二个线性层 `self.w_2` 中，并返回其结果。

整体而言，这段代码实现了位置前馈神经网络层。它通过两个线性层和激活函数（ReLU）实现了非线性变换，从而对输入进行维度变换和特征提取。通过随机失活操作，有助于减少过拟合。前馈神经网络层在 Transformer 模型中被用于对序列中的每个位置进行独立的特征映射，增强了模型的表达能力。

## Embeddings and Softmax

Similarly to other sequence transduction models, we use learned
embeddings to convert the input tokens and output tokens to vectors
of dimension $d_{\text{model}}$.  We also use the usual learned
linear transformation and softmax function to convert the decoder
output to predicted next-token probabilities.  In our model, we
share the same weight matrix between the two embedding layers and
the pre-softmax linear transformation, similar to
[(cite)](https://arxiv.org/abs/1608.05859). In the embedding layers,
we multiply those weights by $\sqrt{d_{\text{model}}}$.

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

这段代码定义了一个嵌入层（Embeddings），用于将输入的离散化的标记（例如单词）转换为连续的向量表示，以便在 Transformer 模型中进行处理。

在初始化函数中，`Embeddings` 类接受两个参数：
- `d_model`：表示嵌入向量的维度，即每个标记被转换为的连续向量的维度。
- `vocab`：表示词汇表的大小，即不同标记的数量。

在初始化过程中，通过调用 `nn.Embedding(vocab, d_model)` 创建了一个嵌入层 `self.lut`，它将输入的标记索引映射为对应的嵌入向量。同时，还记录了嵌入向量的维度 `self.d_model`。

在前向传播函数中，输入为 `x`，它是一个表示标记索引的张量。通过 `self.lut(x)` 将输入张量 `x` 输入到嵌入层中，将每个标记索引转换为对应的嵌入向量。最后，将嵌入向量乘以 `math.sqrt(self.d_model)` 进行缩放，以满足 Transformer 模型的数学约束（根据论文中的建议，对嵌入向量进行缩放可以更好地适应模型的训练）。

整体而言，这段代码实现了一个嵌入层，用于将离散化的标记映射为连续的向量表示。嵌入层在 Transformer 模型中起着重要的作用，它将离散的输入转换为连续的表示，捕捉标记之间的语义关系，并提供给后续的模型进行处理和学习。

## Positional Encoding

Since our model contains no recurrence and no convolution, in order
for the model to make use of the order of the sequence, we must
inject some information about the relative or absolute position of
the tokens in the sequence.  To this end, we add "positional
encodings" to the input embeddings at the bottoms of the encoder and
decoder stacks.  The positional encodings have the same dimension
$d_{\text{model}}$ as the embeddings, so that the two can be summed.
There are many choices of positional encodings, learned and fixed
[(cite)](https://arxiv.org/pdf/1705.03122.pdf).

In this work, we use sine and cosine functions of different frequencies:

$$PE_{(pos,2i)} = \sin(pos / 10000^{2i/d_{\text{model}}})$$

$$PE_{(pos,2i+1)} = \cos(pos / 10000^{2i/d_{\text{model}}})$$

where $pos$ is the position and $i$ is the dimension.  That is, each
dimension of the positional encoding corresponds to a sinusoid.  The
wavelengths form a geometric progression from $2\pi$ to $10000 \cdot
2\pi$.  We chose this function because we hypothesized it would
allow the model to easily learn to attend by relative positions,
since for any fixed offset $k$, $PE_{pos+k}$ can be represented as a
linear function of $PE_{pos}$.

In addition, we apply dropout to the sums of the embeddings and the
positional encodings in both the encoder and decoder stacks.  For
the base model, we use a rate of $P_{drop}=0.1$.



In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

这段代码定义了一个位置编码（Positional Encoding）模块，用于为输入序列中的每个位置添加位置信息。

在初始化函数中，`PositionalEncoding` 类接受三个参数：
- `d_model`：表示输入和输出的特征维度。
- `dropout`：表示用于随机失活的概率。
- `max_len`：表示序列的最大长度，默认为 5000。

在初始化过程中，通过计算位置编码，构建了一个形状为 `[max_len, d_model]` 的位置编码张量 `pe`。具体计算方法如下：

1) 创建一个全零张量 `pe`，形状为 `[max_len, d_model]`，其中 `max_len` 是序列的最大长度，`d_model` 是特征维度。

2) 创建一个形状为 `[max_len, 1]` 的位置张量 `position`，其中每个元素表示序列中的位置索引。

3) 创建一个形状为 `[d_model/2]` 的除法项 `div_term`，其中每个元素计算为 `exp(-log(10000) * 2i / d_model)`，其中 `i` 表示元素索引。

4) 使用 `torch.sin` 和 `torch.cos` 函数计算正弦和余弦值，并将它们分别应用于位置张量乘以除法项的结果。通过广播机制，将正弦和余弦值分别填充到位置编码张量的奇数和偶数列上。

5) 最后，通过 `unsqueeze(0)` 在位置编码张量的最前面添加一个维度，以匹配输入张量的批次维度。

在前向传播函数中，输入为 `x`，表示经过嵌入层处理后的张量。将位置编码张量 `self.pe` 的前 `x.size(1)` 个位置编码加到输入张量 `x` 上，并应用随机失活操作。这样，位置编码就会与输入特征相加，提供每个位置的位置信息。

整体而言，这段代码实现了位置编码模块，用于为输入序列添加位置信息。位置编码通过正弦和余弦函数计算位置相关的编码值，并与输入张量相加。位置编码允许模型根据位置信息进行建模，有助于捕捉序列中元素之间的顺序关系。


> Below the positional encoding will add in a sine wave based on
> position. The frequency and offset of the wave is different for
> each dimension.

In [ ]:
def example_positional():
    pe = PositionalEncoding(20, 0)
    y = pe.forward(torch.zeros(1, 100, 20))

    data = pd.concat(
        [
            pd.DataFrame(
                {
                    "embedding": y[0, :, dim],
                    "dimension": dim,
                    "position": list(range(100)),
                }
            )
            for dim in [4, 5, 6, 7]
        ]
    )

    return (
        alt.Chart(data)
        .mark_line()
        .properties(width=800)
        .encode(x="position", y="embedding", color="dimension:N")
        .interactive()
    )


show_example(example_positional)


We also experimented with using learned positional embeddings
[(cite)](https://arxiv.org/pdf/1705.03122.pdf) instead, and found
that the two versions produced nearly identical results.  We chose
the sinusoidal version because it may allow the model to extrapolate
to sequence lengths longer than the ones encountered during
training.

## Full Model

> Here we define a function from hyperparameters to a full model.

In [ ]:
def make_model(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

这段代码定义了一个帮助函数 `make_model`，用于根据给定的超参数构建一个完整的 Transformer 模型。

在函数参数中，传入了以下超参数：
- `src_vocab`：源语言词汇表的大小。
- `tgt_vocab`：目标语言词汇表的大小。
- `N`：编码器和解码器中的层数。
- `d_model`：模型的特征维度。
- `d_ff`：前馈神经网络隐藏层的特征维度。
- `h`：多头注意力中的头数。
- `dropout`：用于随机失活的概率。

在函数实现中，首先进行了 `copy.deepcopy` 操作，用于深拷贝对象，保证模型的参数共享与独立性。

接下来，通过调用 `MultiHeadedAttention` 类创建了多头注意力层 `attn`，并通过调用 `PositionwiseFeedForward` 类创建了位置前馈神经网络层 `ff`，以及通过调用 `PositionalEncoding` 类创建了位置编码层 `position`。

然后，通过将以上创建的模块组合在一起，构建了一个完整的 Transformer 模型 `model`。该模型由以下组件组成：
- 编码器（`Encoder`）：由 `N` 个相同的编码器层（`EncoderLayer`）堆叠而成。
- 解码器（`Decoder`）：由 `N` 个相同的解码器层（`DecoderLayer`）堆叠而成。
- 源语言嵌入层和位置编码层：通过调用 `Embeddings` 类和 `PositionalEncoding` 类分别创建。
- 目标语言嵌入层和位置编码层：通过调用 `Embeddings` 类和 `PositionalEncoding` 类分别创建。
- 生成器（`Generator`）：用于生成目标语言词汇的线性变换层。

最后，通过遍历模型的参数，使用 Glorot / fan_avg 初始化方法（也称为 Xavier 初始化方法）对模型的参数进行初始化。

总结起来，这段代码通过给定的超参数构建了一个完整的 Transformer 模型。该模型包括编码器、解码器、嵌入层、位置编码层和生成器。这个函数提供了一种方便的方式来创建 Transformer 模型，并使用 Glorot / fan_avg 初始化方法对参数进行初始化，以确保模型在训练过程中的有效性和准确性。

## Inference:

> Here we make a forward step to generate a prediction of the
model. We try to use our transformer to memorize the input. As you
will see the output is randomly generated due to the fact that the
model is not trained yet. In the next tutorial we will build the
training function and try to train our model to memorize the numbers
from 1 to 10.

In [ ]:
def inference_test():
    test_model = make_model(11, 11, 2)
    test_model.eval()
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    src_mask = torch.ones(1, 1, 10)

    memory = test_model.encode(src, src_mask)
    ys = torch.zeros(1, 1).type_as(src)

    for i in range(9):
        out = test_model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = test_model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )

    print("Example Untrained Model Prediction:", ys)


def run_tests():
    for _ in range(10):
        inference_test()


show_example(run_tests)

这段代码包含了两个函数 `inference_test()` 和 `run_tests()`，用于进行模型的推理测试。

函数 `inference_test()`：
- 创建一个测试模型 `test_model`，通过调用 `make_model(11, 11, 2)` 创建一个模型实例，其中源语言和目标语言的词汇表大小都为 11，层数为 2。
- 将模型设为评估模式，即调用 `test_model.eval()`。
- 定义一个源语言序列 `src`，其形状为 `[1, 10]`，内容为 `[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]`。
- 定义源语言掩码 `src_mask`，形状为 `[1, 1, 10]`，全为 1。
- 使用模型的 `encode` 方法将源语言序列 `src` 编码为 `memory`。
- 初始化目标语言序列 `ys`，形状为 `[1, 1]`，元素全为 0，与源语言序列的数据类型相同。
- 进行循环遍历，迭代次数为 9，用于生成目标语言序列。
  - 使用模型的 `decode` 方法对 `memory` 进行解码，得到解码结果 `out`。
  - 使用解码结果 `out` 的最后一个时间步的输出通过模型的生成器（输出层）得到概率分布 `prob`。
  - 使用 `torch.max` 函数找到概率最高的单词，并将其作为下一个时间步的输入单词。
  - 将下一个单词添加到目标语言序列 `ys` 中。
- 打印输出结果，即模型的预测结果。

函数 `run_tests()`：
- 进行 10 次 `inference_test()` 的循环运行，用于多次测试模型的推理能力。

最后，通过调用 `show_example(run_tests)` 函数来展示运行测试的结果。

总体而言，这段代码用于测试模型的推理过程。通过给定的源语言序列，使用训练好的模型进行解码，生成目标语言序列的预测结果，并打印输出。通过多次运行 `inference_test()` 函数来进行推理测试，以验证模型的预测能力。

# Part 2: Model Training

# Training

This section describes the training regime for our models.


> We stop for a quick interlude to introduce some of the tools
> needed to train a standard encoder decoder model. First we define a
> batch object that holds the src and target sentences for training,
> as well as constructing the masks.

## Batches and Masking

In [ ]:
class Batch:
    """Object for holding a batch of data with mask during training."""

    def __init__(self, src, tgt=None, pad=2):  # 2 = <blank>
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            self.tgt = tgt[:, :-1]
            self.tgt_y = tgt[:, 1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y != pad).data.sum()

    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask

这段代码定义了一个名为 `Batch` 的类，用于在训练过程中存储一个数据批次及其相应的掩码。

在类的初始化函数中，接受三个参数：
- `src`：源语言数据序列。
- `tgt`：目标语言数据序列（可选）。
- `pad`：用于填充的特殊符号的索引（默认为 2）。

在初始化过程中，执行以下操作：
- 将源语言数据序列 `src` 存储到 `self.src` 中。
- 通过比较 `src` 和 `pad` 的值，创建源语言数据序列的掩码 `self.src_mask`。掩码将非填充符号设为 1，填充符号设为 0，并在倒数第二个维度上添加一个维度，以与数据序列的维度对齐。

如果目标语言数据序列 `tgt` 不为空，则还执行以下操作：
- 将目标语言数据序列 `tgt` 去除最后一个时间步的标记，存储到 `self.tgt` 中。
- 将目标语言数据序列 `tgt` 去除第一个时间步的标记，存储到 `self.tgt_y` 中。这样可以获得目标语言数据序列的标签。
- 通过调用 `make_std_mask` 静态方法创建目标语言数据序列的掩码 `self.tgt_mask`。该方法用于创建一个掩码，用于隐藏填充符号和未来的单词。首先将非填充符号设为 1，然后通过调用 `subsequent_mask` 函数创建一个掩码，将未来的单词设为 0。最后，将两个掩码按位与运算，得到最终的目标语言数据序列掩码。
- 使用 `(self.tgt_y != pad).data.sum()` 计算目标语言数据序列中非填充符号的数量，存储到 `self.ntokens` 中。

`make_std_mask` 静态方法用于创建一个掩码，用于隐藏填充符号和未来的单词。它接受两个参数：
- `tgt`：目标语言数据序列。
- `pad`：用于填充的特殊符号的索引。

在方法内部，执行以下操作：
- 将目标语言数据序列中非填充符号设为 1，填充符号设为 0，创建目标语言数据序列的掩码 `tgt_mask`。并在倒数第二个维度上添加一个维度，以与数据序列的维度对齐。
- 通过调用 `subsequent_mask` 函数创建一个掩码，将未来的单词设为 0。
- 将两个掩码按位与运算，得到最终的目标语言数据序列掩码，并返回。

总结起来，这段代码定义了一个 `Batch` 类，用于存储训练过程中的数据批次及其相应的掩码。通过类的初始化函数，可以根据源语言数据序列和目标语言数据序列创建批次对象，并生成相应的掩码。这个类提供了方便的方法来处理数据批次，并生成目标语言数据序列的掩码。


> Next we create a generic training and scoring function to keep
> track of loss. We pass in a generic loss compute function that
> also handles parameter updates.

## Training Loop

In [ ]:
class TrainState:
    """Track number of steps, examples, and tokens processed"""

    step: int = 0  # Steps in the current epoch
    accum_step: int = 0  # Number of gradient accumulation steps
    samples: int = 0  # total # of examples used
    tokens: int = 0  # total # of tokens processed

这段代码定义了一个名为 `TrainState` 的类，用于跟踪训练过程中处理的步数、样本数和标记（tokens）数。

类中定义了以下属性：
- `step`：当前训练周期内的步数。
- `accum_step`：梯度累积的步数。
- `samples`：使用的总样本数。
- `tokens`：处理的总标记数。

这些属性用于记录训练过程中的统计信息，例如用于确定当前训练周期的步数、梯度累积的步数、处理的样本数和标记数。通过访问这些属性，可以追踪训练过程中的进度和性能指标。

需要注意的是，这段代码使用了 Python 3.6+ 的类型注解语法，声明了属性的类型。属性的默认值在这里都被设置为整数类型，初始值为 0。

In [ ]:
def run_epoch(
    data_iter,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode="train",
    accum_iter=1,
    train_state=TrainState(),
):
    """Train a single epoch"""
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
        # loss_node = loss_node / accum_iter
        if mode == "train" or mode == "train+log":
            loss_node.backward()
            train_state.step += 1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
                train_state.accum_step += 1
            scheduler.step()

        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            )
            start = time.time()
            tokens = 0
        del loss
        del loss_node
    return total_loss / total_tokens, train_state

这段代码定义了一个名为 `run_epoch` 的函数，用于训练一个周期（epoch）的模型。

函数接受以下参数：
- `data_iter`：数据迭代器，用于生成训练数据批次。
- `model`：模型实例，用于进行前向计算。
- `loss_compute`：损失计算对象，用于计算损失。
- `optimizer`：优化器，用于更新模型参数。
- `scheduler`：学习率调度器，用于调整学习率。
- `mode`：模式，可选值为 `"train"`、`"train+log"` 或 `"eval"`，表示当前的运行模式，默认为 `"train"`。
- `accum_iter`：梯度累积的迭代次数，默认为 1，表示不进行梯度累积。
- `train_state`：训练状态对象，用于记录训练过程中的统计信息。

函数内部执行以下操作：
- 初始化一些变量，包括 `start`（开始时间）、`total_tokens`（总标记数）和 `total_loss`（总损失）等。
- 迭代遍历数据迭代器，获取数据批次。
- 调用模型的前向计算方法 `forward`，传入批次数据，得到模型的输出 `out`。
- 调用损失计算对象的 `loss_compute` 方法，传入模型输出、目标语言序列和标记数，计算损失和损失节点（用于梯度反向传播）。
- 如果当前模式是训练模式（"train" 或 "train+log"）：
  - 执行损失节点的反向传播。
  - 更新训练状态中的步数、样本数和标记数等统计信息。
  - 如果达到了梯度累积的迭代次数（`i % accum_iter == 0`）：
    - 执行优化器的参数更新。
    - 清空梯度，以便进行下一轮的累积梯度计算。
    - 更新训练状态中的梯度累积步数。
  - 执行学习率调度器的更新。
- 累加总损失和总标记数。
- 更新当前周期内处理的标记数。
- 如果满足打印日志的条件（`i % 40 == 1`），且当前模式是训练模式（"train" 或 "train+log"）：
  - 获取当前优化器的学习率。
  - 计算经过一定时间间隔的标记速度。
  - 打印日志，包括当前周期的步数、梯度累积步数、损失、标记速度和学习率等信息。
- 清空损失和损失节点，释放内存。
- 返回总损失除以总标记数得到的平均损失，以及更新后的训练状态对象。

总结起来，这段代码实现了一个训练周期的函数 `run_epoch`，通过循环迭代数据批次，进行模型的前向计算、损失计算、梯度反向传播和参数更新等操作。同时，根据指定的模式和梯度累积设置，记录训练过程中的统计信息，并打印训练日志。

## Training Data and Batching

We trained on the standard WMT 2014 English-German dataset
consisting of about 4.5 million sentence pairs.  Sentences were
encoded using byte-pair encoding, which has a shared source-target
vocabulary of about 37000 tokens. For English-French, we used the
significantly larger WMT 2014 English-French dataset consisting of
36M sentences and split tokens into a 32000 word-piece vocabulary.


Sentence pairs were batched together by approximate sequence length.
Each training batch contained a set of sentence pairs containing
approximately 25000 source tokens and 25000 target tokens.

## Hardware and Schedule

We trained our models on one machine with 8 NVIDIA P100 GPUs.  For
our base models using the hyperparameters described throughout the
paper, each training step took about 0.4 seconds.  We trained the
base models for a total of 100,000 steps or 12 hours. For our big
models, step time was 1.0 seconds.  The big models were trained for
300,000 steps (3.5 days).

## Optimizer

We used the Adam optimizer [(cite)](https://arxiv.org/abs/1412.6980)
with $\beta_1=0.9$, $\beta_2=0.98$ and $\epsilon=10^{-9}$.  We
varied the learning rate over the course of training, according to
the formula:

$$
lrate = d_{\text{model}}^{-0.5} \cdot
  \min({step\_num}^{-0.5},
    {step\_num} \cdot {warmup\_steps}^{-1.5})
$$

This corresponds to increasing the learning rate linearly for the
first $warmup\_steps$ training steps, and decreasing it thereafter
proportionally to the inverse square root of the step number.  We
used $warmup\_steps=4000$.


> Note: This part is very important. Need to train with this setup
> of the model.


> Example of the curves of this model for different model sizes and
> for optimization hyperparameters.

In [ ]:
def rate(step, model_size, factor, warmup):
    """
    we have to default the step to 1 for LambdaLR function
    to avoid zero raising to negative power.
    """
    if step == 0:
        step = 1
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

这段代码定义了一个函数 `rate`，用于计算学习率调度器的学习率。

函数接受以下参数：
- `step`：当前的训练步数。
- `model_size`：模型的大小。
- `factor`：缩放因子，用于调整学习率的比例。
- `warmup`：预热步数，用于控制学习率的预热阶段。

函数内部执行以下操作：
- 首先，对 `step` 进行检查，如果为 0，则将其设置为 1，以避免零的负幂运算。
- 计算学习率的公式，其中：
  - `model_size ** (-0.5)` 是模型大小的负平方根，用于缩放学习率。
  - `min(step ** (-0.5), step * warmup ** (-1.5))` 是两项中的最小值，用于控制学习率在预热阶段和后续阶段的变化。
    - 在预热阶段，学习率随着步数的增加以步数的负平方根的速度减小。
    - 在后续阶段，学习率随着步数的增加以步数的负平方根的速度减小，但相对于预热阶段，减小的速度较慢。

最后，将缩放因子与计算得到的学习率乘积作为最终的学习率返回。

这段代码实现了一个基于模型大小、当前步数和预热阶段的学习率计算方法，用于在训练过程中动态调整学习率的大小。

In [ ]:
def example_learning_schedule():
    opts = [
        [512, 1, 4000],  # example 1
        [512, 1, 8000],  # example 2
        [256, 1, 4000],  # example 3
    ]

    dummy_model = torch.nn.Linear(1, 1)
    learning_rates = []

    # we have 3 examples in opts list.
    for idx, example in enumerate(opts):
        # run 20000 epoch for each example
        optimizer = torch.optim.Adam(
            dummy_model.parameters(), lr=1, betas=(0.9, 0.98), eps=1e-9
        )
        lr_scheduler = LambdaLR(
            optimizer=optimizer, lr_lambda=lambda step: rate(step, *example)
        )
        tmp = []
        # take 20K dummy training steps, save the learning rate at each step
        for step in range(20000):
            tmp.append(optimizer.param_groups[0]["lr"])
            optimizer.step()
            lr_scheduler.step()
        learning_rates.append(tmp)

    learning_rates = torch.tensor(learning_rates)

    # Enable altair to handle more than 5000 rows
    alt.data_transformers.disable_max_rows()

    opts_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Learning Rate": learning_rates[warmup_idx, :],
                    "model_size:warmup": ["512:4000", "512:8000", "256:4000"][
                        warmup_idx
                    ],
                    "step": range(20000),
                }
            )
            for warmup_idx in [0, 1, 2]
        ]
    )

    return (
        alt.Chart(opts_data)
        .mark_line()
        .properties(width=600)
        .encode(x="step", y="Learning Rate", color="model_size:warmup:N")
        .interactive()
    )


example_learning_schedule()

这段代码展示了一个学习率的示例调度表。它使用了不同的模型大小和预热步数组合来生成学习率的变化曲线，并使用 Altair 可视化库绘制了学习率随训练步数的变化。

代码的执行流程如下：
- 定义了一个包含 3 个示例的列表 `opts`，每个示例包含模型大小、缩放因子和预热步数。
- 创建了一个名为 `dummy_model` 的线性模型实例，用于演示目的。
- 定义了一个空列表 `learning_rates`，用于存储不同示例下的学习率变化。
- 对于每个示例，进行如下操作：
  - 创建一个 Adam 优化器，并设置初始学习率、动量参数和 epsilon。
  - 创建一个学习率调度器 `lr_scheduler`，使用 LambdaLR 类，传入优化器和学习率计算函数 `lr_lambda`，其中 `lr_lambda` 调用了前面定义的 `rate` 函数，参数是当前步数和示例中的模型大小、缩放因子和预热步数。
  - 创建临时列表 `tmp`，用于存储每个训练步骤的学习率。
  - 进行 20000 个虚拟的训练步骤：
    - 将当前学习率添加到临时列表 `tmp` 中。
    - 执行优化器的参数更新。
    - 执行学习率调度器的更新。
  - 将临时列表 `tmp` 添加到 `learning_rates` 列表中。
- 将 `learning_rates` 转换为 PyTorch 的张量。
- 使用 Altair 可视化库，创建一个折线图，展示学习率随训练步数的变化。图表的 x 轴为步数，y 轴为学习率，颜色区分不同模型大小和预热步数的组合。
- 返回生成的可视化图表。

这段代码的目的是通过可视化展示不同模型大小和预热步数对学习率调度的影响，帮助理解学习率在训练过程中的变化情况。

---
`torch.optim.Adam` 和 `torch.optim.SGD` 是 PyTorch 中常用的优化器类，用于在训练神经网络时更新模型的参数。

主要的差别如下：

1. **更新策略**：
   - Adam（Adaptive Moment Estimation）：Adam 是一种自适应学习率优化算法，结合了动量梯度下降和RMSprop的思想。它使用梯度的一阶矩估计（均值）和二阶矩估计（方差）来自适应地调整学习率。
   - SGD（Stochastic Gradient Descent）：SGD 是随机梯度下降的简称，它根据每个样本的梯度更新模型参数。SGD 更新的方向是在当前样本的梯度方向上，所以它具有一定的随机性。

2. **动量**：
   - Adam：Adam 使用动量来加速梯度下降，并在更新时考虑了梯度的一阶矩估计（动量）。
   - SGD：SGD 也可以使用动量来加速收敛，通过在更新时引入上一步的更新方向的权重。这有助于克服局部最小值，并在参数空间中更快地搜索。

3. **学习率调度**：
   - Adam：Adam 使用自适应的学习率调度，根据参数梯度的估计值来自适应地调整学习率。
   - SGD：SGD 的学习率通常是预先指定的，并且可以在训练过程中使用学习率调度器进行动态调整。

总的来说，Adam 适用于大多数情况，并且在许多任务和网络架构中表现良好。它通常能够更快地收敛，并且对学习率的选择更加鲁棒。然而，对于某些特定的问题和网络，SGD 也可能表现出色，特别是当使用合适的学习率调度策略和动量设置时。在实践中，选择优化器通常需要根据具体问题和实验来进行调整和比较。

---
`LambdaLR` 是 PyTorch 中的一个学习率调度器类，用于根据自定义的函数调整优化器的学习率。它可以根据训练的当前步数或 epoch 数量来动态地改变学习率。

`LambdaLR` 的作用是根据用户定义的 `lr_lambda` 函数对每个优化器的学习率进行调整。该函数接受当前的步数或 epoch 数量作为输入，并返回相应的学习率因子。学习率因子会乘以初始学习率，从而得到调整后的学习率。

使用 `LambdaLR` 可以实现各种学习率调度策略，例如：
- 线性学习率衰减：随着训练步数或 epoch 的增加，逐渐降低学习率。
- 指数学习率衰减：学习率按指数衰减，例如每个 epoch 减少一定的倍数。
- 阶梯学习率调整：在特定的步数或 epoch 处更改学习率，例如进行一些预定的学习率衰减或增加操作。

使用示例：
```python
# 创建优化器和模型
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# 创建 LambdaLR 调度器，定义学习率调度函数
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda step: 0.95 ** step)

# 在训练循环中更新学习率
for epoch in range(num_epochs):
    for step, batch in enumerate(data_loader):
        # 执行优化器的参数更新
        optimizer.step()

        # 执行学习率调度器的更新
        lr_scheduler.step()
```

通过自定义 `lr_lambda` 函数，可以根据需要灵活地调整学习率。可以根据当前的步数或 epoch 数量制定自己的学习率调度策略，以优化训练过程并改善模型的收敛效果。

## Regularization

### Label Smoothing

During training, we employed label smoothing of value
$\epsilon_{ls}=0.1$ [(cite)](https://arxiv.org/abs/1512.00567).
This hurts perplexity, as the model learns to be more unsure, but
improves accuracy and BLEU score.


> We implement label smoothing using the KL div loss. Instead of
> using a one-hot target distribution, we create a distribution that
> has `confidence` of the correct word and the rest of the
> `smoothing` mass distributed throughout the vocabulary.

In [ ]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())

上述代码实现了标签平滑（Label Smoothing）的功能。

标签平滑是一种用于改善分类任务中模型的泛化能力的技术。在传统的分类任务中，标签通常是使用独热编码表示的，即目标类别的索引位置为1，其它位置为0。然而，独热编码的标签在训练过程中可能导致模型过于自信和过拟合。标签平滑通过将一部分置信度分配给非目标类别，来减少模型的过拟合。

该代码中的 `LabelSmoothing` 类继承自 `nn.Module`，它的作用是计算带有标签平滑的交叉熵损失。以下是代码的详细解析：

- `__init__(self, size, padding_idx, smoothing=0.0)`：初始化函数，接受以下参数：
  - `size`：标签的类别数量。
  - `padding_idx`：填充索引，用于忽略填充标签的损失计算。
  - `smoothing`：标签平滑参数，用于控制非目标类别的置信度。

- `forward(self, x, target)`：前向传播函数，接受输入 `x` 和目标标签 `target`，返回计算得到的标签平滑损失。
  - `x` 是模型的输出结果，通常是经过 softmax 激活函数处理后的预测概率。
  - `target` 是真实的目标标签。

在前向传播过程中，首先通过 `x.data.clone()` 创建一个与 `x` 形状相同的张量 `true_dist`，并将所有元素初始化为标签平滑参数除以类别数量减去2（即目标类别和填充类别之外的类别数量）。然后，使用 `scatter_` 方法将真实目标标签的索引位置处设置为标签平滑参数的另一个部分，从而分配置信度给目标类别。同时，将填充类别的置信度设置为0。接下来，通过 `torch.nonzero` 找到目标标签中为填充类别的位置，并将 `true_dist` 中对应的位置置为0。最后，使用 `nn.KLDivLoss` 计算模型预测的概率分布 `x` 和标签平滑后的真实分布 `true_dist` 之间的 KL 散度损失。

通过使用 `LabelSmoothing` 类，可以将标签平滑应用于分类任务的损失计算中，从而提升模型的泛化性能。

---

代码中的`target.data.unsqueeze(1)` 是将 `target.data` 张量在维度1上进行扩展操作。

在PyTorch中，`unsqueeze(dim)` 方法用于在指定的维度上增加一个维度。参数 `dim` 指定了要插入的新维度的索引位置。

具体到这段代码中，`target.data` 是一个一维的张量，形状为 `[batch_size]`，其中每个元素表示一个样本的目标标签。调用 `unsqueeze(1)` 后，会在维度1上插入一个新的维度，使其形状变为 `[batch_size, 1]`。也就是说，每个目标标签都会变成一个包含单个元素的一维张量。

这样做的目的是为了与 `true_dist` 张量进行相应的索引操作，使其能够在正确的位置上分配标签平滑参数。在代码中，`true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)` 使用 `target.data.unsqueeze(1)` 作为索引，在维度1上将 `self.confidence` 的值分配给相应的位置。

通过使用 `unsqueeze(1)`，可以确保目标标签的形状与 `true_dist` 张量的形状一致，以便进行正确的索引操作。

---

`true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)` 是对 `true_dist` 张量进行按索引散射（scatter）操作。

在 PyTorch 中，`scatter_(dim, index, src)` 方法用于根据索引 `index` 将源张量 `src` 的值散射到目标张量中的指定维度 `dim` 上的指定位置。下划线（`_`）表示该方法是原地操作，会修改原始张量。

具体到这段代码中，`true_dist` 是一个形状为 `[batch_size, num_classes]` 的张量，表示标签平滑后的真实分布。`target.data.unsqueeze(1)` 是形状为 `[batch_size, 1]` 的索引张量，表示每个样本的目标标签。

通过调用 `scatter_` 方法，将 `self.confidence` 的值散射到 `true_dist` 张量的维度1上的指定位置，使得每个样本的目标标签处的位置获得更高的置信度。

具体操作如下：
- `dim=1` 表示在维度1上进行散射操作，即对每个样本的目标标签位置进行操作。
- `index=target.data.unsqueeze(1)` 表示使用 `target.data.unsqueeze(1)` 作为索引，指定要散射的位置。
- `src=self.confidence` 表示使用 `self.confidence` 的值作为散射的源，即要散射到目标张量的值。

综上所述，`true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)` 将 `self.confidence` 的值按照目标标签的索引位置，散射到 `true_dist` 张量的相应位置上，以形成标签平滑后的真实分布。

---

`nn.KLDivLoss` 是用于计算 KL 散度损失（Kullback-Leibler Divergence Loss）的函数。KL 散度是衡量两个概率分布之间差异的指标。

在这段代码中，`nn.KLDivLoss` 的输入是模型预测的概率分布 `x` 和标签平滑后的真实分布 `true_dist`。它会计算这两个分布之间的 KL 散度，并返回对应的损失值。

KL 散度衡量了模型预测分布相对于真实分布的差异程度。当模型的预测分布与真实分布完全一致时，KL 散度为0。当两个分布差异较大时，KL 散度的值会较大。

在标签平滑的情况下，真实分布进行了平滑处理，以减少模型过于自信或过于绝对的预测。通过计算模型预测分布和平滑后的真实分布之间的 KL 散度损失，可以引导模型更加准确地拟合平滑后的目标分布，从而提高模型的泛化能力和鲁棒性。

因此，在这段代码中，`nn.KLDivLoss` 的计算目的是为了最小化模型预测分布与标签平滑后的真实分布之间的差异，从而使模型更好地适应平滑后的目标分布。


> Here we can see an example of how the mass is distributed to the
> words based on confidence.

In [ ]:
# Example of label smoothing.


def example_label_smoothing():
    crit = LabelSmoothing(5, 0, 0.4)
    predict = torch.FloatTensor(
        [
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
        ]
    )
    crit(x=predict.log(), target=torch.LongTensor([2, 1, 0, 3, 3]))
    print(crit.true_dist)
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "target distribution": crit.true_dist[x, y].flatten(),
                    "columns": y,
                    "rows": x,
                }
            )
            for y in range(5)
            for x in range(5)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect(color="Blue", opacity=1)
        .properties(height=200, width=200)
        .encode(
            alt.X("columns:O", title=None),
            alt.Y("rows:O", title=None),
            alt.Color(
                "target distribution:Q", scale=alt.Scale(scheme="viridis")
            ),
        )
        .interactive()
    )


show_example(example_label_smoothing)

这段代码演示了标签平滑（label smoothing）的例子。

首先，定义了一个 `LabelSmoothing` 类，用于实现标签平滑的操作。该类继承自 `nn.Module`，其中的 `forward` 方法实现了标签平滑的计算逻辑。

在 `example_label_smoothing` 函数中，首先创建了一个 `LabelSmoothing` 对象 `crit`，指定了类别数为5，填充索引为0，平滑因子为0.4。

接下来，创建了一个形状为 `[5, 5]` 的预测张量 `predict`，其中包含了5个样本的预测概率分布。

然后，调用 `crit` 对象的 `forward` 方法，传入预测张量的对数概率 `predict.log()` 和目标标签张量 `torch.LongTensor([2, 1, 0, 3, 3])`，进行标签平滑的计算。

接着，通过使用 `pd.concat` 组合了多个 DataFrame，生成了一个包含目标分布数据的 DataFrame `LS_data`。每行代表一个样本，每列代表一个类别，目标分布的值表示目标类别的置信度。

最后，使用 `alt.Chart` 创建了一个矩形图，以可视化目标分布的结果。不同的颜色表示不同的目标分布值，从深到浅表示分布值的大小。

综上所述，该代码展示了标签平滑操作后的目标分布的可视化结果，通过颜色来表示每个位置上的目标分布值的大小。


> Label smoothing actually starts to penalize the model if it gets
> very confident about a given choice.

In [ ]:
def loss(x, crit):
    d = x + 3 * 1
    predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d]])
    return crit(predict.log(), torch.LongTensor([1])).data


def penalization_visualization():
    crit = LabelSmoothing(5, 0, 0.1)
    loss_data = pd.DataFrame(
        {
            "Loss": [loss(x, crit) for x in range(1, 100)],
            "Steps": list(range(99)),
        }
    ).astype("float")

    return (
        alt.Chart(loss_data)
        .mark_line()
        .properties(width=350)
        .encode(
            x="Steps",
            y="Loss",
        )
        .interactive()
    )


show_example(penalization_visualization)

这段代码实现了一个用于可视化惩罚项的函数。首先，定义了一个 `loss` 函数，它接受一个参数 `x` 和一个 `crit`（`LabelSmoothing` 类的实例），并返回模型的损失值。

在 `loss` 函数中，通过对输入 `x` 进行一系列计算，生成了一个预测的概率分布 `predict`。然后，将该预测分布和一个标签值 `[1]` 传递给 `crit` 对象进行计算。其中，标签值 `[1]` 表示目标类别的索引。

在 `penalization_visualization` 函数中，使用了一个 `LabelSmoothing` 对象 `crit` 和一组不同的 `x` 值，通过调用 `loss` 函数计算了对应的损失值。然后，将损失值和相应的步骤数放入一个数据框 `loss_data` 中。

最后，使用 `alt.Chart` 创建一个图表，将步骤数作为 x 轴，损失值作为 y 轴进行可视化。通过这个图表可以观察到，随着输入 `x` 的增加，损失值逐渐增加，表明在使用标签平滑时，模型的预测与目标之间的差异增大，从而引入了一定的惩罚效果。

该图表的目的是可视化标签平滑的惩罚效果，帮助理解在不同输入情况下，惩罚项对模型的损失值的影响。

# A First  Example

> We can begin by trying out a simple copy-task. Given a random set
> of input symbols from a small vocabulary, the goal is to generate
> back those same symbols.

## Synthetic Data

In [ ]:
def data_gen(V, batch_size, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.randint(1, V, size=(batch_size, 10))
        data[:, 0] = 1
        src = data.requires_grad_(False).clone().detach()
        tgt = data.requires_grad_(False).clone().detach()
        yield Batch(src, tgt, 0)

这段代码定义了一个数据生成器函数 `data_gen`，用于生成用于源-目标复制任务的随机数据。

函数接受三个参数：
- `V`：表示词汇表的大小，数据中随机生成的单词将在范围 `[1, V)` 内取值。
- `batch_size`：表示每个批次的样本数量。
- `nbatches`：表示生成的批次数量。

在生成数据的过程中，每个批次会生成一个大小为 `batch_size` 的随机数据张量 `data`，形状为 `(batch_size, 10)`。其中，数据的每一行表示一个序列，包括 10 个单词。

为了模拟源-目标复制任务，将每个序列的第一个单词设置为 1，表示起始标记。这是为了让模型学会复制源序列中的其余单词到目标序列中。

然后，使用 `.requires_grad_(False).clone().detach()` 对生成的数据张量进行浅拷贝，并设置 `requires_grad` 属性为 `False`，使得数据张量不参与梯度计算。这是因为在生成数据时，我们只需要输入和目标数据，而不需要计算其梯度。

最后，使用 `Batch` 类将源数据 `src` 和目标数据 `tgt` 封装成一个批次对象，并使用 `yield` 语句将该批次返回，从而实现了数据的生成器功能。

调用该生成器函数可以获得一系列包含源数据和目标数据的批次对象，用于训练或评估模型。

## Loss Computation

In [ ]:
class SimpleLossCompute:
    "A simple loss compute and train function."

    def __init__(self, generator, criterion):
        self.generator = generator
        self.criterion = criterion

    def __call__(self, x, y, norm):
        x = self.generator(x)
        sloss = (
            self.criterion(
                x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)
            )
            / norm
        )
        return sloss.data * norm, sloss

这段代码定义了一个简单的损失计算和训练函数 `SimpleLossCompute`。

该函数接受两个参数：
- `generator`：一个生成器模型，用于将输入 `x` 转换为预测结果。
- `criterion`：损失函数，用于计算预测结果与目标 `y` 之间的损失。

函数的 `__call__` 方法定义了损失计算和训练的逻辑。它接受三个参数：
- `x`：预测结果，经过生成器模型生成的张量。
- `y`：目标标签，表示真实值的张量。
- `norm`：归一化系数，用于对损失进行归一化。

在函数内部，首先将预测结果 `x` 输入到生成器模型中，得到最终的预测值。然后，通过调用损失函数 `criterion` 计算预测结果 `x` 和目标标签 `y` 之间的损失。注意，为了方便计算，需要对 `x` 和 `y` 进行形状调整，将其转换为二维张量。最后，将损失除以归一化系数 `norm` 进行归一化。

函数返回两个值：
- `sloss.data * norm`：未经归一化的损失值，乘以 `norm` 后表示实际损失。
- `sloss`：归一化后的损失值。

该函数主要用于简化损失计算和训练过程的代码，使代码更加清晰和易读。可以根据实际需求进行修改和扩展。

---

`x.contiguous().view(-1, x.size(-1))` 的含义是将张量 `x` 进行连续化操作（contiguous）并改变其形状。

在PyTorch中，有些操作要求输入张量是连续的（contiguous），即内存中元素的存储是连续的。而有些操作可能会导致张量的存储变得不连续，这时需要使用 `contiguous` 方法将其转为连续的存储。

`.view(-1, x.size(-1))` 的作用是将张量 `x` 的形状调整为 `(batch_size * seq_len, num_classes)`，其中 `batch_size` 表示批量大小，`seq_len` 表示序列长度，`num_classes` 表示类别数量。这个操作会保持张量中元素的顺序不变，只是将张量进行重新排列，以便进行后续的损失计算。

总而言之，`x.contiguous().view(-1, x.size(-1))` 将张量 `x` 进行连续化操作，并将其形状调整为二维张量，方便进行损失计算和其他操作。

## Greedy Decoding

> This code predicts a translation using greedy decoding for simplicity.

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys

这段代码实现了贪婪解码（Greedy Decoding）的过程，用于生成模型的预测序列。

输入参数包括：
- `model`：要使用的模型
- `src`：输入序列
- `src_mask`：输入序列的掩码
- `max_len`：生成序列的最大长度
- `start_symbol`：起始符号的索引

函数首先通过模型的编码器将输入序列编码为一个表示记忆（memory）。然后，初始化输出序列 `ys`，只包含起始符号。接下来，通过循环逐步生成序列的每个元素，直到达到最大长度。在每个时间步，模型的解码器根据记忆、输入序列的掩码和当前生成的序列 `ys` 预测下一个时间步的输出。通过模型的生成器（generator），将解码器输出的最后一个时间步的结果转换为概率分布。然后，选择概率最高的单词作为下一个时间步的输出，并将其添加到输出序列 `ys` 中。循环直到生成序列达到最大长度为止。

最终，函数返回生成的序列 `ys`。

总结来说，`greedy_decode` 函数利用 Transformer 模型对输入序列进行贪婪解码，生成预测序列。

In [ ]:
# Train the simple copy task.
def example_simple_model():
    V = 11
    criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
    model = make_model(V, V, N=2)

    optimizer = torch.optim.Adam(
        model.parameters(), lr=0.5, betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, model_size=model.src_embed[0].d_model, factor=1.0, warmup=400
        ),
    )

    batch_size = 80
    for epoch in range(20):
        model.train()
        run_epoch(
            data_gen(V, batch_size, 20),
            model,
            SimpleLossCompute(model.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train",
        )
        model.eval()
        run_epoch(
            data_gen(V, batch_size, 5),
            model,
            SimpleLossCompute(model.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )[0]

    model.eval()
    src = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
    max_len = src.shape[1]
    src_mask = torch.ones(1, 1, max_len)
    print(greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0))


#execute_example(example_simple_model)

这段代码演示了如何训练一个简单的复制任务（copy task）并使用训练好的模型进行预测。

首先，定义了一个大小为 11 的标签平滑（Label Smoothing）损失函数 `criterion`，和一个由 `make_model` 创建的 Transformer 模型 `model`。然后，定义了优化器 `optimizer` 和学习率调度器 `lr_scheduler`。

接下来，进入训练循环，共进行 20 个周期（epochs）。在每个周期中，模型分为训练模式和评估模式。调用 `run_epoch` 函数对训练数据进行训练或评估，并传入相应的损失计算对象、优化器和学习率调度器。训练模式下使用真实的优化器和学习率调度器进行参数更新，评估模式下使用虚拟的优化器和学习率调度器仅进行损失计算。训练模式中，每个周期的训练数据使用 `data_gen` 生成。

训练完成后，将模型切换为评估模式，并使用 `greedy_decode` 函数对输入序列进行贪婪解码，生成预测序列。

最终，打印输出预测序列。

总结来说，这段代码展示了如何使用 Transformer 模型训练和预测一个简单的复制任务，并展示了训练过程中的损失和预测结果。

# Part 3: A Real World Example

> Now we consider a real-world example using the Multi30k
> German-English Translation task. This task is much smaller than
> the WMT task considered in the paper, but it illustrates the whole
> system. We also show how to use multi-gpu processing to make it
> really fast.

## Data Loading

> We will load the dataset using torchtext and spacy for
> tokenization.

In [ ]:
# Load spacy tokenizer models, download them if they haven't been
# downloaded already


def load_tokenizers():

    try:
        spacy_de = spacy.load("de_core_news_sm")
    except IOError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_de, spacy_en

这段代码用于加载 spaCy 分词器模型。如果模型尚未下载，它将自动下载所需的模型。

首先，尝试加载 "de_core_news_sm" 的 spaCy 分词器模型。如果模型加载失败（即抛出 IOError 异常），则通过执行命令 `python -m spacy download de_core_news_sm` 下载并安装所需的模型，然后再次尝试加载模型。

接下来，尝试加载 "en_core_web_sm" 的 spaCy 分词器模型。如果模型加载失败，同样执行命令 `python -m spacy download en_core_web_sm` 下载并安装所需的模型，然后再次尝试加载模型。

最后，返回加载成功的 spaCy 分词器模型对象 `spacy_de` 和 `spacy_en`。

这段代码确保在使用 spaCy 分词器之前，必要的模型已经下载和加载。

---

变量 `spacy_de` 和 `spacy_en` 是 spaCy 分词器模型对象。它们是经过训练和加载的自然语言处理模型，可以用于对德语（`spacy_de`）和英语（`spacy_en`）进行分词操作。

这些对象具有各种方法和属性，可以用于执行文本处理任务，如分词、词性标注、命名实体识别等。通过调用这些对象的方法，可以将文本输入模型，然后获得相应的分词结果或其他文本处理结果。

例如，可以使用以下方式调用分词器模型对象进行分词操作：

```python
text = "This is an example sentence."
tokens = spacy_en(text)
```

在上述示例中，`spacy_en` 是加载的英语分词器模型对象，`text` 是待分词的文本。通过调用 `spacy_en` 对象，将文本传递给它，就可以获得对应的分词结果 `tokens`。

需要注意的是，这里的 `spacy_de` 和 `spacy_en` 是根据代码中的变量命名推测的对象名称，实际命名可以根据代码的上下文而有所不同。

In [ ]:
def tokenize(text, tokenizer):
    return [tok.text for tok in tokenizer.tokenizer(text)]


def yield_tokens(data_iter, tokenizer, index):
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])

这段代码包含两个函数：`tokenize` 和 `yield_tokens`。

1. `tokenize(text, tokenizer)` 函数接受一个文本字符串 `text` 和一个分词器 `tokenizer` 作为输入，并返回该文本的分词结果。它通过遍历 `tokenizer.tokenizer(text)` 得到的分词对象列表，提取每个分词对象的 `text` 属性，将其作为分词结果返回。

2. `yield_tokens(data_iter, tokenizer, index)` 函数是一个生成器函数，它接受一个数据迭代器 `data_iter`、一个分词器 `tokenizer` 和一个索引 `index` 作为输入。它通过遍历 `data_iter` 中的元素（假设每个元素是一个包含多个字段的元组），获取索引 `index` 对应的字段，然后将该字段的文本输入到分词器 `tokenizer` 中进行分词。生成器函数使用 `yield` 关键字产生每个分词结果，使得可以逐步获取分词结果而不需要一次性加载全部数据。

这两个函数可以结合使用，例如：


In [ ]:
spacy_de, spacy_en = load_tokenizers()

text = "This is an example sentence."
tokens = tokenize(text, spacy_en)
print(tokens)
data_iter = [("Sentence 1", "Sentence 2"), ("Sentence 3", "Sentence 4")]
token_generator = yield_tokens(data_iter, spacy_en, index=0)
for tokens in token_generator:
    print(tokens)

在上述示例中，我们首先加载了英语分词器模型 `spacy_en`，然后使用 `tokenize` 函数将文本进行分词。接下来，我们定义了一个数据迭代器 `data_iter`，其中包含了一些元组数据。通过调用 `yield_tokens` 函数生成一个分词器的生成器，并逐步获取分词结果。

In [ ]:
def build_vocabulary(spacy_de, spacy_en):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    print("Building German Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_de, index=0),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    print("Building English Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_en, index=1),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"])

    return vocab_src, vocab_tgt


def load_vocab(spacy_de, spacy_en):
    if not exists("vocab.pt"):
        vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
        torch.save((vocab_src, vocab_tgt), "vocab.pt")
    else:
        vocab_src, vocab_tgt = torch.load("vocab.pt")
    print("Finished.\nVocabulary sizes:")
    print(len(vocab_src))
    print(len(vocab_tgt))
    return vocab_src, vocab_tgt


if is_interactive_notebook():
    # global variables used later in the script
    spacy_de, spacy_en = show_example(load_tokenizers)
    vocab_src, vocab_tgt = show_example(load_vocab, args=[spacy_de, spacy_en])

这段代码包含两个函数：`build_vocabulary` 和 `load_vocab`，以及一些全局变量的设置。

1. `build_vocabulary(spacy_de, spacy_en)` 函数用于构建德语和英语的词汇表。它首先定义了两个内部函数 `tokenize_de` 和 `tokenize_en`，分别用于对德语和英语文本进行分词。接下来，它使用 `datasets.Multi30k` 函数加载 Multi30k 数据集的训练集、验证集和测试集。然后，分别针对德语和英语文本，调用 `yield_tokens` 函数生成分词器的生成器，并通过 `build_vocab_from_iterator` 函数构建词汇表。该函数还设置了一些特殊的标记（`<s>`、`</s>`、`<blank>`、`<unk>`）以及最小频率参数。最后，将默认索引设置为 `<unk>` 标记，并返回德语和英语的词汇表对象。

2. `load_vocab(spacy_de, spacy_en)` 函数用于加载词汇表。它首先检查是否已存在名为 "vocab.pt" 的文件，如果不存在，则调用 `build_vocabulary` 函数构建词汇表，并将结果保存到 "vocab.pt" 文件中。如果文件已存在，则直接加载词汇表。最后，打印词汇表的大小，并返回德语和英语的词汇表对象。

在代码的最后部分，如果当前环境是交互式笔记本，则展示了 `load_tokenizers` 和 `load_vocab` 函数的示例运行，并将结果赋值给全局变量 `spacy_de` 和 `spacy_en`、`vocab_src` 和 `vocab_tgt`。

这些函数的目的是准备语言模型所需的分词器和词汇表。分词器用于将文本拆分为单词或子词，而词汇表包含了模型所需的单词或子词的索引和相关信息。

---

**关于 Multi30K**

Multi30k 是一个用于多语言机器翻译任务的数据集。它是在 WMT 2016 评测活动中发布的，用于英语到德语的翻译任务。该数据集源自于 Flickr 图像共享平台上的图像描述任务，每个图像都有多个对应的英语和德语描述。

Multi30k 数据集包含了约 3 万个图像，共计约 29,000 个训练样本、1,014 个验证样本和 1,000 个测试样本。图像和对应的描述被组织成多个标签文件，每个文件都包含了图像的 ID、英语描述和德语描述。

该数据集的主要目标是让模型学习从图像到对应语言的翻译关系。因此，对于机器翻译任务，我们可以将图像描述作为源语言输入，将目标语言描述作为目标输出，训练模型来完成翻译任务。

Multi30k 数据集是自然语言处理和机器翻译领域常用的数据集之一，用于评估不同模型在跨语言翻译任务上的性能。Multi30k 数据集包含来自多个来源的英语-德语平行句子，用于图像描述任务。每个样本包含一张图像和对应的英语和德语描述。这些描述涵盖了各种主题和语言风格。该数据集广泛用于机器翻译和图像字幕生成等自然语言处理任务的研究和评估

---
**关于 vocab.pt**

`vocab.pt` 是一个保存了源语言和目标语言的词汇表的文件。它使用 PyTorch 的 `torch.save()` 函数进行保存。保存的文件格式是 PyTorch 的序列化格式，其中包含了两个对象：`vocab_src` 和 `vocab_tgt`，分别表示源语言和目标语言的词汇表。

你可以使用 `torch.load()` 函数加载 `vocab.pt` 文件，并获得源语言和目标语言的词汇表对象。

---
**关于 词汇表**

`vocab_src` 和 `vocab_tgt` 是两个词汇表对象，它们都是 `torchtext.vocab.Vocab` 类的实例。这个类表示一个词汇表，包含了单词到索引的映射以及相关的属性和方法。

词汇表对象包含以下主要属性和方法：
- `stoi`：单词到索引的字典，可以通过单词来获取其对应的索引。
- `itos`：索引到单词的列表，可以通过索引来获取对应的单词。
- `freqs`：单词的频率统计信息，包含每个单词在数据集中出现的次数。
- `unk_index`：表示未知单词的索引。
- `pad_index`：表示填充单词的索引。
- `bos_index`：表示序列起始的特殊标记（如`<s>`）的索引。
- `eos_index`：表示序列结束的特殊标记（如`</s>`）的索引。

可以使用这些属性和方法来对词汇表进行查询和操作，例如查找单词的索引、查找索引对应的单词、获取词汇表的大小等。



> Batching matters a ton for speed. We want to have very evenly
> divided batches, with absolutely minimal padding. To do this we
> have to hack a bit around the default torchtext batching. This
> code patches their default batching to make sure we search over
> enough sentences to find tight batches.

## Iterators

In [ ]:
def collate_batch(
    batch,
    src_pipeline,
    tgt_pipeline,
    src_vocab,
    tgt_vocab,
    device,
    max_padding=128,
    pad_id=2,
):
    bs_id = torch.tensor([0], device=device)  # <s> token id
    eos_id = torch.tensor([1], device=device)  # </s> token id
    src_list, tgt_list = [], []
    for (_src, _tgt) in batch:
        processed_src = torch.cat(
            [
                bs_id,
                torch.tensor(
                    src_vocab(src_pipeline(_src)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    tgt_vocab(tgt_pipeline(_tgt)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        src_list.append(
            # warning - overwrites values for negative values of padding - len
            pad(
                processed_src,
                (
                    0,
                    max_padding - len(processed_src),
                ),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_padding - len(processed_tgt)),
                value=pad_id,
            )
        )

    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)
    return (src, tgt)

这段代码实现了一个`collate_batch`函数，用于将一个批次的数据进行处理和填充。

函数接受以下参数：
- `batch`：一个批次的数据，其中每个元素是一个源语言句子和目标语言句子的元组。
- `src_pipeline`：源语言数据的处理管道函数，用于对源语言句子进行预处理操作。
- `tgt_pipeline`：目标语言数据的处理管道函数，用于对目标语言句子进行预处理操作。
- `src_vocab`：源语言词汇表对象，用于将源语言句子转换为索引序列。
- `tgt_vocab`：目标语言词汇表对象，用于将目标语言句子转换为索引序列。
- `device`：指定数据存储的设备（如CPU或GPU）。
- `max_padding`：填充后的序列最大长度。
- `pad_id`：填充标记的索引。

函数的主要步骤如下：
1. 对于批次中的每个源语言句子和目标语言句子：
   - 使用相应的处理管道函数进行预处理，将句子转换为索引序列。
   - 添加起始标记和结束标记，并将其与源语言词汇表和目标语言词汇表转换为Tensor对象。
   - 将处理后的序列进行填充，使其达到指定的最大长度。
2. 将填充后的源语言序列和目标语言序列转换为Tensor对象，并将它们堆叠成一个Tensor张量作为返回值。

该函数的作用是将批次的原始文本数据转换为填充后的Tensor数据，以便输入到模型进行训练或推理。

In [ ]:
def create_dataloaders(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    batch_size=12000,
    max_padding=128,
    is_distributed=True,
):
    # def create_dataloaders(batch_size=12000):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
        )

    train_iter, valid_iter, test_iter = datasets.Multi30k(
        language_pair=("de", "en")
    )

    train_iter_map = to_map_style_dataset(
        train_iter
    )  # DistributedSampler needs a dataset len()
    train_sampler = (
        DistributedSampler(train_iter_map) if is_distributed else None
    )
    valid_iter_map = to_map_style_dataset(valid_iter)
    valid_sampler = (
        DistributedSampler(valid_iter_map) if is_distributed else None
    )

    train_dataloader = DataLoader(
        train_iter_map,
        batch_size=batch_size,
        shuffle=(train_sampler is None),
        sampler=train_sampler,
        collate_fn=collate_fn,
    )
    valid_dataloader = DataLoader(
        valid_iter_map,
        batch_size=batch_size,
        shuffle=(valid_sampler is None),
        sampler=valid_sampler,
        collate_fn=collate_fn,
    )
    return train_dataloader, valid_dataloader

这段代码实现了一个`create_dataloaders`函数，用于创建数据加载器（`DataLoader`）对象，用于批量加载和处理训练、验证和测试数据。

函数接受以下参数：
- `device`：指定数据存储的设备（如CPU或GPU）。
- `vocab_src`：源语言词汇表对象。
- `vocab_tgt`：目标语言词汇表对象。
- `spacy_de`：用于德语文本分词的SpaCy模型对象。
- `spacy_en`：用于英语文本分词的SpaCy模型对象。
- `batch_size`：批次大小，默认为12000。
- `max_padding`：填充后的序列最大长度，默认为128。
- `is_distributed`：是否使用分布式训练，默认为True。

函数的主要步骤如下：
1. 定义用于将德语文本分词的`tokenize_de`函数和将英语文本分词的`tokenize_en`函数。
2. 定义`collate_fn`函数，用于将一个批次的原始文本数据转换为填充后的Tensor数据。
3. 使用`datasets.Multi30k`加载训练、验证和测试数据集，并将其转换为Map-style的数据集（`to_map_style_dataset`函数将Iterable-style的数据集转换为Map-style的数据集，需要传入一个具有`__getitem__`方法的对象）。
4. 根据是否使用分布式训练，创建训练数据加载器（`train_dataloader`）和验证数据加载器（`valid_dataloader`），并指定批次大小、是否洗牌和采样器（如果使用分布式训练）以及数据的处理函数（`collate_fn`）。
5. 返回训练数据加载器和验证数据加载器。

该函数的作用是将原始文本数据加载为经过处理和填充的Tensor数据，并创建对应的数据加载器，方便在训练过程中以批次的形式输入到模型中。

---

**关于to_map_style_dataset**

`to_map_style_dataset(train_iter)` 的作用是将原始的迭代式数据集对象 `train_iter` 转换为 Map-style 数据集对象 `train_iter_map`。

在 PyTorch 中，存在两种类型的数据集对象：Iterable-style 数据集和 Map-style 数据集。

- Iterable-style 数据集：这种数据集对象是基于迭代器的，通过迭代访问数据集中的样本。它没有固定的长度，每次迭代返回一个样本。
- Map-style 数据集：这种数据集对象是基于索引的，可以通过索引或键来访问数据集中的样本。它具有固定的长度，并且可以直接通过索引访问指定位置的样本。

`to_map_style_dataset` 函数的作用就是将 Iterable-style 数据集对象转换为 Map-style 数据集对象。这样做的目的是为了能够使用 `DataLoader` 加载器更方便地处理数据集，因为 `DataLoader` 通常要求输入的数据集对象是 Map-style 的。

在给定的代码中，`train_iter` 是 Multi30k 数据集的 Iterable-style 数据集对象。通过调用 `to_map_style_dataset(train_iter)` 将其转换为 Map-style 数据集对象 `train_iter_map`，以便后续能够使用 `DataLoader` 来加载、迭代和处理训练数据集。

--- 

**关于DataLoader**
`DataLoader`是PyTorch中的一个实用类，用于将数据集封装成可迭代的批量加载器。它在训练过程中提供了数据的批量加载、数据打乱、并行加速等功能，简化了数据处理的流程。

调用`DataLoader`的作用是将数据集划分为小批次并提供对数据的迭代访问。具体而言，`DataLoader`的主要作用包括：

1. 数据加载：将原始数据集加载到内存中，并对数据进行处理和预处理（如分词、填充、编码等）。
2. 批次划分：将数据划分为小批次，每个批次包含一定数量的样本。
3. 数据打乱：可选择在每个训练周期之前对数据进行打乱，以增加模型的训练稳定性和泛化能力。
4. 并行加速：通过使用多个工作线程（`num_workers`参数）并行加载数据，加快数据加载速度，提高训练效率。
5. 数据预取：在训练过程中，可以异步预取下一个批次的数据，使模型的训练过程更加流畅。

通过调用`DataLoader`，我们可以轻松地对训练、验证和测试数据进行批量加载和处理，从而更高效地训练和评估模型。

`train_dataloader = DataLoader(...)` 主要目的是创建一个用于训练的数据加载器（`train_dataloader`），它将训练数据集划分为小批次，并提供对数据的迭代访问。

具体而言，上述代码的参数和设置含义如下：

- `train_iter_map`: 训练数据集，已经通过`to_map_style_dataset`转换为Map-style数据集，即可通过索引访问样本。
- `batch_size`: 每个批次的样本数量。
- `shuffle`: 是否在每个训练周期之前对数据进行打乱。如果`train_sampler`为`None`，则默认为`True`，即每个周期都对数据进行打乱。
- `sampler`: 数据采样器，用于定义样本的抽样策略。在分布式训练中，使用`DistributedSampler`对数据进行分布式抽样，确保每个进程获取不同的样本。
- `collate_fn`: 用于对每个批次中的样本进行处理和预处理的函数。在这里，使用`collate_batch`函数对样本进行填充、编码等操作。

通过使用`DataLoader`创建训练数据加载器，可以方便地迭代访问训练数据集的小批次样本，实现高效的模型训练。`DataLoader`会根据设定的参数和设置自动处理数据的加载、打乱和批次划分，简化了数据处理的过程，提高了训练效率。

## Training the System

In [ ]:
def train_worker(
    gpu,
    ngpus_per_node,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    config,
    is_distributed=False,
):
    print(f"Train worker process using GPU: {gpu} for training", flush=True)
    torch.cuda.set_device(gpu)

    pad_idx = vocab_tgt["<blank>"]
    d_model = 512
    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.cuda(gpu)
    module = model
    is_main_process = True
    if is_distributed:
        dist.init_process_group(
            "nccl", init_method="env://", rank=gpu, world_size=ngpus_per_node
        )
        model = DDP(model, device_ids=[gpu])
        module = model.module
        is_main_process = gpu == 0

    criterion = LabelSmoothing(
        size=len(vocab_tgt), padding_idx=pad_idx, smoothing=0.1
    )
    criterion.cuda(gpu)

    train_dataloader, valid_dataloader = create_dataloaders(
        gpu,
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=config["batch_size"] // ngpus_per_node,
        max_padding=config["max_padding"],
        is_distributed=is_distributed,
    )

    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["base_lr"], betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, d_model, factor=1, warmup=config["warmup"]
        ),
    )
    train_state = TrainState()

    for epoch in range(config["num_epochs"]):
        if is_distributed:
            train_dataloader.sampler.set_epoch(epoch)
            valid_dataloader.sampler.set_epoch(epoch)

        model.train()
        print(f"[GPU{gpu}] Epoch {epoch} Training ====", flush=True)
        _, train_state = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train+log",
            accum_iter=config["accum_iter"],
            train_state=train_state,
        )

        GPUtil.showUtilization()
        if is_main_process:
            file_path = "%s%.2d.pt" % (config["file_prefix"], epoch)
            torch.save(module.state_dict(), file_path)
        torch.cuda.empty_cache()

        print(f"[GPU{gpu}] Epoch {epoch} Validation ====", flush=True)
        model.eval()
        sloss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in valid_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )
        print(sloss)
        torch.cuda.empty_cache()

    if is_main_process:
        file_path = "%sfinal.pt" % config["file_prefix"]
        torch.save(module.state_dict(), file_path)

这段代码是一个用于训练的工作进程函数 `train_worker()`。它接收一些参数，并执行训练过程。

首先，它设置了当前 GPU 设备，并将模型移动到该设备上。如果启用了分布式训练，还会使用 `torch.distributed.init_process_group()` 初始化分布式进程组，并使用 `torch.nn.parallel.DistributedDataParallel()` 将模型包装为分布式数据并行模型。

接下来，它创建了损失函数 `criterion`，使用了 `LabelSmoothing` 类定义的标签平滑损失函数，并将其移动到 GPU 设备上。

然后，它调用 `create_dataloaders()` 函数创建训练集和验证集的数据加载器。这些数据加载器负责将数据集分成批次，并在训练过程中提供批次数据供模型使用。

接着，它创建了优化器 `optimizer`，这里使用了 Adam 优化器，并设置了学习率、动量等参数。

然后，它创建了学习率调度器 `lr_scheduler`，使用 `LambdaLR` 类定义的学习率调度器，根据预定义的学习率调度函数进行学习率的更新。

接下来，它定义了训练状态 `train_state`，用于跟踪训练过程中的步数、样本数和标记数。

接下来，它开始进行训练的循环，循环遍历预定义的训练轮数 `config["num_epochs"]`。

在每个训练轮次中，首先设置数据加载器的 epoch，以确保每个 GPU 都使用相同的数据。然后将模型设置为训练模式，并调用 `run_epoch()` 函数执行一个训练周期的训练过程。

在训练过程中，它打印训练信息，并保存模型的参数到文件中。然后清空 GPU 缓存，释放不再需要的内存。

接着，它将模型设置为评估模式，并调用 `run_epoch()` 函数执行一个训练周期的评估过程，计算验证集上的损失。

最后，在主进程中保存最终的模型参数到文件中。

整个训练过程中，会根据配置的设定进行分布式训练和 GPU 之间的通信与同步。

该代码段是一个多 GPU 训练的示例，每个 GPU 设备都运行一个工作进程，共同参与模型的训练和参数更新，以提高训练效率和速度。

---

**关于 model.train()**

语句 `model.train()` 的作用是将模型设置为训练模式。在训练模式下，模型会启用一些特定的行为，如启用 Batch Normalization 和 Dropout 层的计算，并记录梯度以进行参数更新。

在训练模式下，模型的 `forward()` 方法会根据输入数据计算模型的输出，并返回相应的结果。同时，模型会保留计算图以进行反向传播和梯度计算。

在深度学习训练过程中，通常在训练阶段使用 `model.train()` 将模型设置为训练模式，以确保模型在训练期间按预期进行计算和参数更新。

---

**关于验证模式的参数**

在验证模式下，我们不需要进行参数更新，因此将优化器（`Optimizer`）和学习率调度器（`Scheduler`）设置为虚拟（dummy）对象是一种常见做法。

在验证模式中，我们只关心模型在验证集上的性能评估，而不需要通过优化器来更新模型的参数。因此，将优化器和学习率调度器设置为虚拟对象可以避免不必要的计算和参数更新。

通过将优化器和学习率调度器设置为虚拟对象，我们可以在验证过程中仅关注模型的前向传播和损失计算，而无需进行反向传播和参数更新的计算，从而提高了验证过程的效率。这样可以减少显存的占用并加快验证的速度。

In [ ]:
def train_distributed_model(vocab_src, vocab_tgt, spacy_de, spacy_en, config):
    ngpus = torch.cuda.device_count()
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12356"
    print(f"Number of GPUs detected: {ngpus}")
    print("Spawning training processes ...")
    mp.spawn(
        train_worker,
        nprocs=ngpus,
        args=(ngpus, vocab_src, vocab_tgt, spacy_de, spacy_en, config, True),
    )


def train_model(vocab_src, vocab_tgt, spacy_de, spacy_en, config):
    if config["distributed"]:
        train_distributed_model(
            vocab_src, vocab_tgt, spacy_de, spacy_en, config
        )
    else:
        train_worker(
            0, 1, vocab_src, vocab_tgt, spacy_de, spacy_en, config, False
        )


def load_trained_model():
    config = {
        "batch_size": 32,
        "distributed": False,
        "num_epochs": 8,
        "accum_iter": 10,
        "base_lr": 1.0,
        "max_padding": 72,
        "warmup": 3000,
        "file_prefix": "multi30k_model_",
    }
    model_path = "multi30k_model_final.pt"
    if not exists(model_path):
        train_model(vocab_src, vocab_tgt, spacy_de, spacy_en, config)

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(torch.load("multi30k_model_final.pt"))
    return model


if is_interactive_notebook():
    model = load_trained_model()

该代码片段涉及训练和加载已训练模型的过程。

1. `train_distributed_model()`: 这是一个用于在分布式设置下进行训练的函数。它通过调用`mp.spawn()`来生成多个训练进程，每个进程在一个GPU上执行`train_worker()`函数。

2. `train_model()`: 这是一个用于在单机设置下进行训练的函数。它直接调用`train_worker()`函数，将GPU设备编号设置为0，表示在单个GPU上进行训练。

3. `load_trained_model()`: 这是一个加载已训练模型的函数。它首先检查是否存在已训练模型的文件。如果模型文件不存在，则调用`train_model()`函数进行训练。然后，它创建一个新的模型对象，使用`make_model()`函数构建模型结构，并从模型文件中加载训练好的参数。

4. 在交互式笔记本环境中，调用`load_trained_model()`函数来加载已训练模型，并将其赋值给变量`model`。

综上所述，这段代码实现了训练和加载模型的功能。它根据配置参数决定是进行分布式训练还是单机训练，并提供了加载已训练模型的选项。


> Once trained we can decode the model to produce a set of
> translations. Here we simply translate the first sentence in the
> validation set. This dataset is pretty small so the translations
> with greedy search are reasonably accurate.

# Additional Components: BPE, Search, Averaging


> So this mostly covers the transformer model itself. There are four
> aspects that we didn't cover explicitly. We also have all these
> additional features implemented in
> [OpenNMT-py](https://github.com/opennmt/opennmt-py).




> 1) BPE/ Word-piece: We can use a library to first preprocess the
> data into subword units. See Rico Sennrich's
> [subword-nmt](https://github.com/rsennrich/subword-nmt)
> implementation. These models will transform the training data to
> look like this:

▁Die ▁Protokoll datei ▁kann ▁ heimlich ▁per ▁E - Mail ▁oder ▁FTP
▁an ▁einen ▁bestimmte n ▁Empfänger ▁gesendet ▁werden .


> 2) Shared Embeddings: When using BPE with shared vocabulary we can
> share the same weight vectors between the source / target /
> generator. See the [(cite)](https://arxiv.org/abs/1608.05859) for
> details. To add this to the model simply do this:

In [ ]:
if False:
    model.src_embed[0].lut.weight = model.tgt_embeddings[0].lut.weight
    model.generator.lut.weight = model.tgt_embed[0].lut.weight


> 3) Beam Search: This is a bit too complicated to cover here. See the
> [OpenNMT-py](https://github.com/OpenNMT/OpenNMT-py/)
> for a pytorch implementation.
>



> 4) Model Averaging: The paper averages the last k checkpoints to
> create an ensembling effect. We can do this after the fact if we
> have a bunch of models:

In [ ]:
def average(model, models):
    "Average models into model"
    for ps in zip(*[m.params() for m in [model] + models]):
        ps[0].copy_(torch.sum(*ps[1:]) / len(ps[1:]))

# Results

On the WMT 2014 English-to-German translation task, the big
transformer model (Transformer (big) in Table 2) outperforms the
best previously reported models (including ensembles) by more than
2.0 BLEU, establishing a new state-of-the-art BLEU score of
28.4. The configuration of this model is listed in the bottom line
of Table 3. Training took 3.5 days on 8 P100 GPUs. Even our base
model surpasses all previously published models and ensembles, at a
fraction of the training cost of any of the competitive models.

On the WMT 2014 English-to-French translation task, our big model
achieves a BLEU score of 41.0, outperforming all of the previously
published single models, at less than 1/4 the training cost of the
previous state-of-the-art model. The Transformer (big) model trained
for English-to-French used dropout rate Pdrop = 0.1, instead of 0.3.


![](images/results.png)



> With the addtional extensions in the last section, the OpenNMT-py
> replication gets to 26.9 on EN-DE WMT. Here I have loaded in those
> parameters to our reimplemenation.

In [ ]:
# Load data and model for output checks

In [ ]:
def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt,
    n_examples=15,
    pad_idx=2,
    eos_string="</s>",
):
    results = [()] * n_examples
    for idx in range(n_examples):
        print("\nExample %d ========\n" % idx)
        b = next(iter(valid_dataloader))
        rb = Batch(b[0], b[1], pad_idx)
        greedy_decode(model, rb.src, rb.src_mask, 64, 0)[0]

        src_tokens = [
            vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx
        ]
        tgt_tokens = [
            vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx
        ]

        print(
            "Source Text (Input)        : "
            + " ".join(src_tokens).replace("\n", "")
        )
        print(
            "Target Text (Ground Truth) : "
            + " ".join(tgt_tokens).replace("\n", "")
        )
        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            " ".join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        print("Model Output               : " + model_txt.replace("\n", ""))
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    return results


def run_model_example(n_examples=5):
    global vocab_src, vocab_tgt, spacy_de, spacy_en

    print("Preparing Data ...")
    _, valid_dataloader = create_dataloaders(
        torch.device("cpu"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )

    print("Loading Trained Model ...")

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(
        torch.load("multi30k_model_final.pt", map_location=torch.device("cpu"))
    )

    print("Checking Model Outputs:")
    example_data = check_outputs(
        valid_dataloader, model, vocab_src, vocab_tgt, n_examples=n_examples
    )
    return model, example_data


# execute_example(run_model_example)

## Attention Visualization

> Even with a greedy decoder the translation looks pretty good. We
> can further visualize it to see what is happening at each layer of
> the attention

In [ ]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    "convert a dense matrix to a data frame with row and column indices"
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s"
                % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s"
                % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        # if float(m[r,c]) != 0 and r < max_row and c < max_col],
        columns=["row", "column", "value", "row_token", "col_token"],
    )


def attn_map(attn, layer, head, row_tokens, col_tokens, max_dim=30):
    df = mtx2df(
        attn[0, head].data,
        max_dim,
        max_dim,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        .properties(height=400, width=400)
        .interactive()
    )

In [ ]:
def get_encoder(model, layer):
    return model.encoder.layers[layer].self_attn.attn


def get_decoder_self(model, layer):
    return model.decoder.layers[layer].self_attn.attn


def get_decoder_src(model, layer):
    return model.decoder.layers[layer].src_attn.attn


def visualize_layer(model, layer, getter_fn, ntokens, row_tokens, col_tokens):
    # ntokens = last_example[0].ntokens
    attn = getter_fn(model, layer)
    n_heads = attn.shape[1]
    charts = [
        attn_map(
            attn,
            0,
            h,
            row_tokens=row_tokens,
            col_tokens=col_tokens,
            max_dim=ntokens,
        )
        for h in range(n_heads)
    ]
    assert n_heads == 8
    return alt.vconcat(
        charts[0]
        # | charts[1]
        | charts[2]
        # | charts[3]
        | charts[4]
        # | charts[5]
        | charts[6]
        # | charts[7]
        # layer + 1 due to 0-indexing
    ).properties(title="Layer %d" % (layer + 1))

## Encoder Self Attention

In [ ]:
def viz_encoder_self():
    model, example_data = run_model_example(n_examples=1)
    example = example_data[
        len(example_data) - 1
    ]  # batch object for the final example

    layer_viz = [
        visualize_layer(
            model, layer, get_encoder, len(example[1]), example[1], example[1]
        )
        for layer in range(6)
    ]
    return alt.hconcat(
        layer_viz[0]
        # & layer_viz[1]
        & layer_viz[2]
        # & layer_viz[3]
        & layer_viz[4]
        # & layer_viz[5]
    )


show_example(viz_encoder_self)

## Decoder Self Attention

In [ ]:
def viz_decoder_self():
    model, example_data = run_model_example(n_examples=1)
    example = example_data[len(example_data) - 1]

    layer_viz = [
        visualize_layer(
            model,
            layer,
            get_decoder_self,
            len(example[1]),
            example[1],
            example[1],
        )
        for layer in range(6)
    ]
    return alt.hconcat(
        layer_viz[0]
        & layer_viz[1]
        & layer_viz[2]
        & layer_viz[3]
        & layer_viz[4]
        & layer_viz[5]
    )


show_example(viz_decoder_self)

## Decoder Src Attention

In [ ]:
def viz_decoder_src():
    model, example_data = run_model_example(n_examples=1)
    example = example_data[len(example_data) - 1]

    layer_viz = [
        visualize_layer(
            model,
            layer,
            get_decoder_src,
            max(len(example[1]), len(example[2])),
            example[1],
            example[2],
        )
        for layer in range(6)
    ]
    return alt.hconcat(
        layer_viz[0]
        & layer_viz[1]
        & layer_viz[2]
        & layer_viz[3]
        & layer_viz[4]
        & layer_viz[5]
    )


show_example(viz_decoder_src)

# Conclusion

 Hopefully this code is useful for future research. Please reach
 out if you have any issues.


 Cheers,
 Sasha Rush, Austin Huang, Suraj Subramanian, Jonathan Sum, Khalid Almubarak,
 Stella Biderman